# 🧠 Neuro_Doc_Assistant — Демонстрация и исследование

## 🎯 Цель ноутбука

Этот notebook демонстрирует полную архитектуру проекта **Neuro_Doc_Assistant** — RAG-системы с детерминированным Agent Controller для работы с корпоративной документацией.

### Что демонстрируется:

1. **Test-First подход**: каждый модуль начинается с формализации требований и тест-кейсов
2. **Agent Controller (FSM)**: детерминированная state machine управляет логикой, а не LLM
3. **Модульная архитектура**: Ingestion → Retrieval → Reranking → Generation → Evaluation
4. **Исследовательский фокус**: метрики Precision@K, влияние chunk_size, overlap, K, reranking

### Чем это отличается от «обычного RAG»:

- ❌ **Обычный RAG**: LLM управляет логикой, «магия в prompt»
- ✅ **Neuro_Doc_Assistant**: 
  - Логика в Agent Controller (FSM)
  - LLM — только исполнитель (embeddings / generation / rerank)
  - Все решения фиксируются в DecisionLog
  - Метрики-driven подход

### Принцип Agent Controller:

```
IDLE → VALIDATE_QUERY → RETRIEVE → METADATA_FILTER → RERANK → GENERATE → VALIDATE_ANSWER → LOG_METRICS → RETURN_RESPONSE
```

Каждое состояние — детерминированное решение, логируемое в DecisionLog.

---

## 📋 Структура ноутбука

0. **Введение** (этот раздел)
1. **Подготовка окружения** — установка библиотек
2. **Данные и домен** — загрузка структуры документов
3. **Формализация UC-1** — тест-кейсы до реализации
4. **Ingestion & Indexing** — DocumentLoader, Chunker, EmbeddingService, QdrantIndexer
5. **Retrieval Layer** — semantic search, параметр K
6. **Reranking** — сравнение с/без reranking
7. **Agent Controller (FSM)** — state machine в действии
8. **Generation Layer** — PromptBuilder, строгая инструкция
9. **Evaluation & Metrics** — Precision@K, сравнение конфигураций
10. **Итоговый вывод** — результаты экспериментов

---

**Примечание**: Этот notebook использует mock-режим для работы без реальных API ключей. Все компоненты работают локально в Colab.


# 1️⃣ Подготовка окружения

Устанавливаем необходимые библиотеки с фиксированными версиями для воспроизводимости.


In [ ]:
# Установка основных зависимостей
!pip install -q tiktoken  # GPT-style BPE токенизация для Chunker (обновлено: без фиксированной версии для совместимости)
!pip install -q qdrant-client>=1.7.0  # Qdrant клиент для векторного поиска (обновлено для совместимости с Qdrant Cloud)
!pip install -q requests>=2.32.5  # HTTP клиент для EmbeddingService (обновлено для совместимости с langchain и datasets)
!pip install -q urllib3>=2.0.2  # Retry стратегия (обновлено: версия 2.0.0 была отозвана из-за бага)
!pip install -q rich  # Красивый вывод в консоль

# Установка зависимостей для RAGAS (реальный режим оценки метрик)
!pip install -q ragas  # Библиотека для оценки качества RAG-систем
!pip install -q langchain-core  # LangChain core для адаптеров RAGAS
!pip install -q langchain-community  # LangChain community компоненты
!pip install -q datasets  # Hugging Face datasets для RAGAS

print("✅ Основные библиотеки установлены")
print("✅ RAGAS и зависимости установлены (реальный режим оценки метрик доступен)")


### Пояснение библиотек:

- **tiktoken**: Токенизация текста для правильного подсчёта размера чанков (GPT-style BPE)
- **qdrant-client**: Клиент для работы с векторной БД Qdrant (в Colab используем in-memory режим)
- **requests**: HTTP-запросы к GigaChat API (в mock-режиме не используется)
- **urllib3**: Retry-стратегия для надёжности запросов
- **rich**: Красивый форматированный вывод для наглядности
- **ragas**: Библиотека для оценки качества RAG-систем (Faithfulness, Answer Relevancy)
- **langchain-core, langchain-community**: LangChain компоненты для адаптеров RAGAS
- **datasets**: Hugging Face datasets для работы с данными в RAGAS


In [ ]:
# Импорты для работы
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
import json
import time
import uuid
import hashlib
import ssl
import urllib3
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import aiohttp
import asyncio
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import print as rprint
from google.colab import userdata

# Отключаем предупреждения о небезопасных SSL запросах (для GigaChat API)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

console = Console()

print("✅ Импорты выполнены")


# 2️⃣ Данные и домен

Загружаем структуру документов с Google Drive и поясняем их назначение.


In [ ]:
# === НАСТРОЙКА GOOGLE DRIVE ===
from google.colab import drive

print("Подключаем Google Диск...")
drive.mount('/content/drive', force_remount=False)

# Путь к данным на Google Drive (адаптируйте под свою структуру)
DRIVE_DATA_PATH = Path("/content/drive/MyDrive/NeuroDoc_Data")
# Локальная рабочая директория
LOCAL_DATA_PATH = Path("/content/neurodoc_data")
LOCAL_DATA_PATH.mkdir(parents=True, exist_ok=True)

# Структура данных проекта
data_structure = {
    "hr/": "HR-документы (политики, регламенты, процедуры)",
    "it/": "IT-документы (SLA сервисов, инструкции, API)",
    "onboarding/": "Документы для онбординга новых сотрудников",
    "compliance/": "Документы по комплаенсу (ГК РФ, ТК РФ, УК РФ)"
}

console.print("\n[bold cyan]Структура данных:[/bold cyan]")
for path, desc in data_structure.items():
    console.print(f"  📁 {path:20s} → {desc}")

# Копируем данные с Google Drive (если они там есть)
import shutil
if DRIVE_DATA_PATH.exists():
    console.print(f"\n[cyan]Копируем данные с Google Drive: {DRIVE_DATA_PATH}[/cyan]")
    for category in data_structure.keys():
        source_dir = DRIVE_DATA_PATH / category.rstrip('/')
        target_dir = LOCAL_DATA_PATH / category.rstrip('/')
        if source_dir.exists():
            shutil.copytree(source_dir, target_dir, dirs_exist_ok=True)
            md_files = list(target_dir.glob("*.md"))
            console.print(f"  ✅ {category}: {len(md_files)} MD файлов")
else:
    console.print(f"\n[yellow]⚠️  Директория {DRIVE_DATA_PATH} не найдена[/yellow]")
    console.print("  Создаём тестовую структуру для демонстрации...")
    for category in data_structure.keys():
        (LOCAL_DATA_PATH / category.rstrip('/')).mkdir(parents=True, exist_ok=True)

console.print("\n[bold yellow]Важно для UC-1:[/bold yellow]")
console.print("  • Документы из hr/ и it/ используются как SLA-источники")
console.print("  • UC-1: 'Какой SLA у сервиса платежей?' — ищет в it/ документах")


### Пояснение документов:

- **HR документы**: Политики удалённой работы, порядок оформления отпуска, больничные, премии, KPI и т.д.
- **IT документы**: SLA сервисов (платежи, API, инфраструктура), инструкции по использованию систем
- **Onboarding**: Руководства для новых сотрудников
- **Compliance**: Юридические документы (ГК РФ, ТК РФ, УК РФ)

**Для UC-1** мы фокусируемся на IT-документах, содержащих информацию о SLA сервисов.


# 3️⃣ Формализация UC-1 (Test-First)

**Принцип Test-First**: сначала формализуем требования и тест-кейсы, затем реализуем.

## Описание UC-1

**Use Case**: Базовый поиск информации по документации

**Сценарий**: Сотрудник задаёт вопрос про SLA сервиса платежей

**Пример запроса**: `"Какой SLA у сервиса платежей?"`

## SLA-ожидания

1. **Precision@3 ≥ 80%**: В топ-3 найденных чанках должно быть ≥ 80% релевантных
2. **Faithfulness ≥ 0.85**: Ответ основан строго на контексте (RAGAS метрика)
3. **Нет галлюцинаций**: Ответ не содержит информации, отсутствующей в источниках
4. **Есть ссылки на источники**: Каждый ответ содержит ссылки на найденные документы


In [ ]:
# Тест-кейсы UC-1 (до реализации)

def test_uc1_requirements():
    """
    Формализация требований UC-1 в виде тест-кейсов.
    
    Эти тесты определяют ожидаемое поведение системы ДО реализации.
    """
    requirements = {
        "test_1_documents_found": {
            "description": "Документы найдены",
            "given": "Документы с описанием SLA загружены и проиндексированы",
            "when": "Пользователь задаёт вопрос про SLA",
            "then": "Система находит релевантные чанки (len(sources) > 0)"
        },
        "test_2_answer_based_on_context": {
            "description": "Ответ опирается на контекст",
            "given": "Найдены релевантные чанки",
            "when": "Генерируется ответ",
            "then": "Ответ содержит ключевые слова из найденных чанков"
        },
        "test_3_no_hallucinations": {
            "description": "Нет ответа вне источников",
            "given": "Найдены чанки с информацией",
            "when": "Генерируется ответ",
            "then": "Текст каждого источника содержится в ответе (case-insensitive)"
        },
        "test_4_precision_at_3": {
            "description": "Precision@3 >= 0.8",
            "given": "Ground truth релевантных чанков определён экспертом",
            "when": "Выполняется retrieval с K=3",
            "then": "Precision@3 >= 0.8 (≥80% релевантных в топ-3)"
        }
    }
    
    return requirements

# Выводим требования
requirements = test_uc1_requirements()
console.print("\n[bold cyan]Тест-кейсы UC-1:[/bold cyan]")
for test_id, test_case in requirements.items():
    console.print(f"\n[bold]{test_id}:[/bold] {test_case['description']}")
    console.print(f"  Given: {test_case['given']}")
    console.print(f"  When: {test_case['when']}")
    console.print(f"  Then: {test_case['then']}")

print("\n✅ Требования формализованы. Теперь можно реализовывать модули.")


# 4️⃣ Ingestion & Indexing (Test-First)

Модули:
1. **DocumentLoader** — загрузка MD файлов
2. **Chunker** — разбиение на чанки с overlap
3. **EmbeddingService** — генерация embeddings (GigaChat / mock)
4. **QdrantIndexer** — индексация в Qdrant (in-memory для Colab)

**Принцип**: Сначала тесты, затем реализация.


In [ ]:
# ============================================
# Модуль 1: DocumentLoader
# ============================================

import re
import uuid
from pathlib import Path

@dataclass
class Document:
    """Представление документа после загрузки."""
    id: str
    text: str
    metadata: dict

class DocumentLoader:
    """
    Загрузчик документов из файловой системы.
    
    Зачем нужна:
    - Загрузка MD файлов из директорий
    - Нормализация текста (удаление лишних пробелов)
    - Извлечение метаданных (category: hr/it/compliance)
    
    Вход: path (str) — путь к файлу или директории
    Выход: List[Document] — список документов с текстом и метаданными
    """
    
    def __init__(self):
        pass
    
    def load_documents(self, path: str) -> List[Document]:
        """Загружает документы из указанного пути."""
        path_obj = Path(path)
        
        if not path_obj.exists():
            raise FileNotFoundError(f"Path does not exist: {path}")
        
        documents = []
        
        if path_obj.is_file():
            if path_obj.suffix.lower() == '.md':
                doc = self._load_single_file(path_obj)
                if doc:
                    documents.append(doc)
        elif path_obj.is_dir():
            md_files = list(path_obj.glob("*.md"))
            for md_file in md_files:
                doc = self._load_single_file(md_file)
                if doc:
                    documents.append(doc)
        else:
            raise ValueError(f"Path is neither file nor directory: {path}")
        
        return documents
    
    def _load_single_file(self, file_path: Path) -> Optional[Document]:
        """Загружает один MD файл."""
        try:
            text = file_path.read_text(encoding='utf-8')
            normalized_text = self._normalize_text(text)
            metadata = self._extract_metadata(file_path)
            doc_id = self._generate_document_id(file_path)
            
            return Document(
                id=doc_id,
                text=normalized_text,
                metadata=metadata
            )
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")
            return None
    
    def _normalize_text(self, text: str) -> str:
        """Нормализация текста: удаление лишних пробелов."""
        # Удаляем множественные пробелы
        text = re.sub(r'\s+', ' ', text)
        # Удаляем пробелы в начале и конце
        text = text.strip()
        return text
    
    def _extract_metadata(self, file_path: Path) -> dict:
        """Извлекает метаданные из пути к файлу."""
        # Определяем категорию по родительской директории
        parent_dir = file_path.parent.name.lower()
        category = parent_dir if parent_dir in ['hr', 'it', 'compliance', 'onboarding'] else 'unknown'
        
        return {
            "category": category,
            "file_path": str(file_path),
            "file_name": file_path.name
        }
    
    def _generate_document_id(self, file_path: Path) -> str:
        """Генерирует уникальный ID документа."""
        return str(uuid.uuid4())

print("✅ DocumentLoader реализован")


In [ ]:
# Тесты для DocumentLoader

def test_document_loader():
    """Тесты для DocumentLoader (pytest-style)."""
    loader = DocumentLoader()
    
    # Тест 1: Загрузка существующего файла
    # В Colab нужно будет загрузить тестовый файл
    # Для демо создадим mock-файл
    test_file = Path("/tmp/test_doc.md")
    test_file.write_text("# Test Document\n\nThis is a test document.", encoding='utf-8')
    
    try:
        docs = loader.load_documents(str(test_file))
        assert len(docs) == 1, "Должен быть загружен один документ"
        assert docs[0].text is not None, "Текст документа не должен быть None"
        assert len(docs[0].text) > 0, "Текст документа не должен быть пустым"
        assert "category" in docs[0].metadata, "Метаданные должны содержать category"
        
        console.print("[green]✅ Тест 1 пройден: Загрузка файла работает[/green]")
    finally:
        if test_file.exists():
            test_file.unlink()
    
    # Тест 2: Нормализация текста
    normalized = loader._normalize_text("  Много    пробелов   здесь  ")
    assert normalized == "Много пробелов здесь", "Нормализация пробелов работает"
    console.print("[green]✅ Тест 2 пройден: Нормализация текста работает[/green]")
    
    # Тест 3: Извлечение метаданных
    test_path = Path("/tmp/hr/test_doc.md")
    metadata = loader._extract_metadata(test_path)
    assert metadata["category"] == "hr", "Категория должна быть 'hr'"
    console.print("[green]✅ Тест 3 пройден: Извлечение метаданных работает[/green]")

test_document_loader()
print("\n✅ Все тесты DocumentLoader пройдены")


In [ ]:
# ============================================
# Модуль 2: Chunker
# ============================================

import tiktoken

@dataclass
class Chunk:
    """Представление чанка документа."""
    chunk_id: str
    doc_id: str
    text: str
    text_length: int
    metadata: dict = field(default_factory=dict)

class Chunker:
    """
    Разбиение документов на чанки с overlap.
    
    Зачем нужна:
    - Разбиение длинных документов на управляемые части
    - Overlap 20-30% для сохранения контекста между чанками
    - Использование токенизации для точного размера
    
    Вход: documents (List[Document]), chunk_size (int), overlap_percent (float)
    Выход: List[Chunk] — список чанков
    
    Ограничения:
    - chunk_size: 200-400 токенов (оптимально 300)
    - overlap_percent: 0.2-0.3 (20-30%)
    """
    
    def __init__(self, encoding_name: str = "cl100k_base"):
        """Инициализация Chunker."""
        try:
            self.encoding = tiktoken.get_encoding(encoding_name)
        except Exception:
            self.encoding = None
    
    def chunk_documents(
        self,
        documents: List[Document],
        chunk_size: int = 300,
        overlap_percent: float = 0.25
    ) -> List[Chunk]:
        """
        Разбивает документы на чанки.
        
        Args:
            documents: Список Document объектов
            chunk_size: Размер чанка в токенах (200-400)
            overlap_percent: Процент overlap между чанками (0.2-0.3)
        """
        all_chunks = []
        
        for doc in documents:
            chunks = self._chunk_single_document(doc, chunk_size, overlap_percent)
            all_chunks.extend(chunks)
        
        return all_chunks
    
    def _chunk_single_document(
        self,
        document: Document,
        chunk_size: int,
        overlap_percent: float
    ) -> List[Chunk]:
        """Разбивает один документ на чанки."""
        text = document.text
        
        # Подсчёт токенов
        text_length = self._count_tokens(text)
        
        # Если текст короткий, возвращаем один чанк
        if text_length <= chunk_size:
            chunk_metadata = document.metadata.copy()
            if "source" not in chunk_metadata:
                chunk_metadata["source"] = chunk_metadata.get("category", "unknown")
            
            return [Chunk(
                chunk_id=f"{document.id}_chunk_000",
                doc_id=document.id,
                text=text,
                text_length=text_length,
                metadata=chunk_metadata
            )]
        
        # Вычисляем overlap в токенах
        overlap_tokens = int(chunk_size * overlap_percent)
        step_size = chunk_size - overlap_tokens
        
        chunks = []
        start = 0
        chunk_index = 0
        
        # Упрощённый подход: разбиваем по символам с учётом приблизительного размера токенов
        # Приблизительно: 1 токен ≈ 3 символа для русского текста
        chars_per_token = 3
        chunk_size_chars = chunk_size * chars_per_token
        overlap_chars = overlap_tokens * chars_per_token
        step_size_chars = step_size * chars_per_token
        
        while start < len(text):
            end = min(start + chunk_size_chars, len(text))
            chunk_text = text[start:end]
            
            chunk_metadata = document.metadata.copy()
            if "source" not in chunk_metadata:
                chunk_metadata["source"] = chunk_metadata.get("category", "unknown")
            
            chunk = Chunk(
                chunk_id=f"{document.id}_chunk_{chunk_index:03d}",
                doc_id=document.id,
                text=chunk_text,
                text_length=self._count_tokens(chunk_text),
                metadata=chunk_metadata
            )
            chunks.append(chunk)
            
            # Сдвигаем окно с учётом overlap
            start += step_size_chars
            chunk_index += 1
        
        return chunks
    
    def _count_tokens(self, text: str) -> int:
        """Подсчитывает количество токенов в тексте."""
        if self.encoding:
            return len(self.encoding.encode(text))
        else:
            # Fallback: приблизительный подсчёт (1 токен ≈ 3 символа)
            return len(text) // 3

print("✅ Chunker реализован")


In [ ]:
# ============================================
# Модуль 3a: GigaChatAuth (OAuth 2.0)
# ============================================

class GigaChatAuth:
    """
    Класс для OAuth 2.0 аутентификации в GigaChat API.
    
    Отвечает за:
    - Получение access token через OAuth 2.0
    - Кэширование токена (действителен 30 минут)
    - Автоматическое обновление токена перед истечением
    """
    
    # Официальные endpoints GigaChat API
    OAUTH_TOKEN_URL = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    
    def __init__(self, auth_key: Optional[str] = None, scope: Optional[str] = None):
        """
        Инициализация GigaChatAuth.
        
        Args:
            auth_key: Base64 encoded "Client ID:Client Secret" (если None, берётся из секретов Colab)
            scope: Scope для OAuth (GIGACHAT_API_PERS, GIGACHAT_API_B2B, GIGACHAT_API_CORP)
        """
        try:
            self.auth_key = auth_key or userdata.get("GIGACHAT_AUTH_KEY")
        except:
            self.auth_key = auth_key
        try:
            self.scope = scope or userdata.get("GIGACHAT_SCOPE", "GIGACHAT_API_PERS")
        except:
            self.scope = scope or "GIGACHAT_API_PERS"
        
        # Кэш токена
        self._access_token_cache: Optional[str] = None
        self._token_expires_at: float = 0
        
        # Настройка сессии с отключенной проверкой SSL
        self.session = requests.Session()
        retry_strategy = Retry(
            total=5,
            backoff_factor=2,
            status_forcelist=[429, 500, 502, 503, 504],
            respect_retry_after_header=True
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)
        self.session.verify = False
    
    def get_access_token(self) -> Optional[str]:
        """Получает access token для GigaChat API."""
        # Проверяем кэш токена
        current_time = time.time()
        if self._access_token_cache and current_time < self._token_expires_at - 60:
            return self._access_token_cache
        
        # Получаем новый токен
        if not self.auth_key:
            return None
        
        rq_uid = str(uuid.uuid4())
        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json",
            "RqUID": rq_uid,
            "Authorization": f"Basic {self.auth_key}"
        }
        data = {"scope": self.scope}
        
        try:
            response = self.session.post(self.OAUTH_TOKEN_URL, headers=headers, data=data, timeout=30)
            if response.status_code == 200:
                response_data = response.json()
                access_token = response_data.get("access_token")
                if access_token:
                    self._access_token_cache = access_token
                    expires_at_ms = response_data.get("expires_at", 0)
                    if expires_at_ms > 1000000000000:
                        self._token_expires_at = expires_at_ms / 1000
                    else:
                        self._token_expires_at = expires_at_ms
                    return access_token
        except Exception as e:
            console.print(f"[yellow]⚠️  Ошибка получения токена: {e}[/yellow]")
            return None
        return None

print("✅ GigaChatAuth реализован")


In [ ]:
# Тесты для Chunker

def test_chunker():
    """Тесты для Chunker."""
    chunker = Chunker()
    
    # Тест 1: Короткий документ (один чанк)
    short_doc = Document(
        id="doc1",
        text="Короткий текст документа.",
        metadata={"category": "hr"}
    )
    chunks = chunker.chunk_documents([short_doc], chunk_size=300)
    assert len(chunks) == 1, "Короткий документ должен быть одним чанком"
    assert chunks[0].text == short_doc.text, "Текст чанка должен совпадать с документом"
    console.print("[green]✅ Тест 1 пройден: Короткий документ обрабатывается корректно[/green]")
    
    # Тест 2: Длинный документ (несколько чанков)
    long_text = " ".join(["Предложение номер"] * 200)  # Длинный текст
    long_doc = Document(
        id="doc2",
        text=long_text,
        metadata={"category": "it"}
    )
    chunks = chunker.chunk_documents([long_doc], chunk_size=100, overlap_percent=0.25)
    assert len(chunks) > 1, "Длинный документ должен быть разбит на несколько чанков"
    console.print(f"[green]✅ Тест 2 пройден: Длинный документ разбит на {len(chunks)} чанков[/green]")
    
    # Тест 3: Overlap между чанками
    if len(chunks) >= 2:
        # Проверяем, что есть overlap (последние слова первого чанка есть в начале второго)
        chunk1_end = chunks[0].text[-50:]
        chunk2_start = chunks[1].text[:50]
        # Упрощённая проверка: есть общие слова
        words1 = set(chunk1_end.split())
        words2 = set(chunk2_start.split())
        common_words = words1.intersection(words2)
        assert len(common_words) > 0, "Должен быть overlap между чанками"
        console.print(f"[green]✅ Тест 3 пройден: Overlap работает ({len(common_words)} общих слов)[/green]")
    
    # Выводим статистику
    console.print(f"\n[cyan]Статистика чанкинга:[/cyan]")
    console.print(f"  Всего чанков: {len(chunks)}")
    console.print(f"  Средний размер чанка: {sum(c.text_length for c in chunks) / len(chunks):.1f} токенов")
    console.print(f"  Размеры: {[c.text_length for c in chunks[:5]]}...")

test_chunker()
print("\n✅ Все тесты Chunker пройдены")


In [ ]:
# ============================================
# Модуль 3: EmbeddingService (GigaChat API)
# ============================================

import numpy as np
import requests
import time
import uuid
import hashlib
from typing import List, Optional
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

class EmbeddingService:
    """
    Сервис для генерации векторных представлений через GigaChat Embeddings API.
    
    Зачем нужна:
    - Генерация embeddings для текстов (для semantic search)
    - В production: GigaChat Embeddings API
    - В Colab: можно использовать mock-режим или реальный API (при наличии ключей)
    
    Вход: texts (List[str])
    Выход: List[List[float]] — список векторов размерности embedding_dim
    
    Ограничения:
    - embedding_dim: 1536 (GigaChat) или 1024
    - batch_size: 10 (для оптимизации)
    """
    
    def __init__(
        self,
        embedding_dim: int = 1536,
        batch_size: int = 10,
        mock_mode: bool = True,
        auth_key: Optional[str] = None,
        scope: Optional[str] = None
    ):
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        
        # Определяем auth_key (из секретов Colab или параметра)
        if not auth_key:
            try:
                auth_key = userdata.get("GIGACHAT_AUTH_KEY")
            except:
                auth_key = None
        
        # Определяем mock mode
        if mock_mode or not auth_key:
            self.mock_mode = True
        else:
            self.mock_mode = False
            self.auth = GigaChatAuth(auth_key=auth_key, scope=scope)
        
        # Настройка HTTP сессии
        self.session = requests.Session()
        retry_strategy = Retry(
            total=5,
            backoff_factor=2,
            status_forcelist=[429, 500, 502, 503, 504],
            respect_retry_after_header=True
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)
        self.session.verify = False
        
        # Официальный endpoint для GigaChat Embeddings API
        self.api_url = "https://gigachat.devices.sberbank.ru/api/v1/embeddings"
    
    def generate_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        Генерирует embeddings для списка текстов.
        
        В mock-режиме: создаёт детерминированные векторы на основе hash.
        В production: вызывает GigaChat Embeddings API.
        """
        if self.mock_mode:
            return [self._generate_mock_embedding(text) for text in texts]
        
        all_embeddings = []
        # Обрабатываем тексты батчами
        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            batch_embeddings = []
            for text in batch:
                embedding = self._call_gigachat_api(text)
                batch_embeddings.append(embedding)
                time.sleep(0.1)  # Задержка между запросами
            all_embeddings.extend(batch_embeddings)
        return all_embeddings
    
    def _call_gigachat_api(self, text: str) -> List[float]:
        """Вызывает GigaChat Embeddings API для одного текста."""
        if self.mock_mode:
            return self._generate_mock_embedding(text)
        
        try:
            access_token = self.auth.get_access_token()
            if not access_token:
                return self._generate_mock_embedding(text)
            
            request_id = str(uuid.uuid4())
            headers = {
                "Authorization": f"Bearer {access_token}",
                "Content-Type": "application/json",
                "X-Request-ID": request_id
            }
            
            payload = {
                "model": "Embeddings",
                "input": text
            }
            
            response = self.session.post(self.api_url, json=payload, headers=headers, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            # Извлекаем embedding из ответа
            embedding = None
            if "data" in data and len(data["data"]) > 0:
                embedding = data["data"][0].get("embedding", [])
            elif "embedding" in data:
                embedding = data["embedding"]
            
            if embedding and len(embedding) in [1024, 1536]:
                self.embedding_dim = len(embedding)  # Обновляем размерность
                # Нормализуем вектор (L2 норма = 1.0) для консистентности
                embedding_array = np.array(embedding)
                norm = np.linalg.norm(embedding_array)
                if norm > 0:
                    embedding_array = embedding_array / norm
                return embedding_array.tolist()
            return self._generate_mock_embedding(text)
        except Exception as e:
            console.print(f"[yellow]⚠️  Ошибка вызова GigaChat Embeddings API: {e}[/yellow]")
            return self._generate_mock_embedding(text)
    
    def _generate_mock_embedding(self, text: str) -> List[float]:
        """Генерирует моковый embedding на основе текста."""
        text_hash = hashlib.md5(text.encode('utf-8')).hexdigest()
        embedding = []
        for i in range(self.embedding_dim):
            hash_index = i % len(text_hash)
            char_value = ord(text_hash[hash_index])
            normalized_value = (char_value % 200 - 100) / 100.0
            embedding.append(normalized_value)
        
        # Нормализуем вектор (L2 норма = 1.0)
        embedding_array = np.array(embedding)
        norm = np.linalg.norm(embedding_array)
        if norm > 0:
            embedding_array = embedding_array / norm
        return embedding_array.tolist()

print("✅ EmbeddingService реализован (поддержка GigaChat API и mock режим)")


In [ ]:
# Тесты для EmbeddingService

def test_embedding_service():
    """Тесты для EmbeddingService."""
    embedding_service = EmbeddingService(embedding_dim=1536, mock_mode=True)
    
    # Тест 1: Генерация embeddings
    texts = ["Текст номер один", "Текст номер два"]
    embeddings = embedding_service.generate_embeddings(texts)
    
    assert len(embeddings) == 2, "Должно быть 2 embeddings"
    assert len(embeddings[0]) == 1536, "Размерность должна быть 1536"
    console.print("[green]✅ Тест 1 пройден: Embeddings генерируются[/green]")
    
    # Тест 2: Нормализация векторов
    vector = np.array(embeddings[0])
    norm = np.linalg.norm(vector)
    assert abs(norm - 1.0) < 0.01, f"Вектор должен быть нормализован (norm={norm:.3f})"
    console.print(f"[green]✅ Тест 2 пройден: Векторы нормализованы (norm={norm:.3f})[/green]")
    
    # Тест 3: Детерминированность (один и тот же текст даёт один и тот же вектор)
    embeddings1 = embedding_service.generate_embeddings(["Тест"])
    embeddings2 = embedding_service.generate_embeddings(["Тест"])
    assert np.allclose(embeddings1[0], embeddings2[0]), "Embeddings должны быть детерминированными"
    console.print("[green]✅ Тест 3 пройден: Embeddings детерминированы[/green]")

test_embedding_service()
print("\n✅ Все тесты EmbeddingService пройдены")


In [ ]:
# ============================================
# Модуль 4: QdrantIndexer (In-Memory для Colab)
# ============================================

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# Получение ключей Qdrant из секретов Colab
QDRANT_URL = None
QDRANT_API_KEY = None

try:
    # Проверяем, доступен ли userdata
    if 'userdata' in globals():
        # Используем второй параметр для значения по умолчанию
        # В Colab userdata.get() может выбрасывать KeyError, поэтому используем try-except
        try:
            QDRANT_URL = userdata.get("QDRANT_URL")
            # Очищаем от пробельных символов (могут быть \r\n в конце)
            if QDRANT_URL:
                QDRANT_URL = QDRANT_URL.strip()
        except (KeyError, AttributeError):
            QDRANT_URL = None
        
        try:
            QDRANT_API_KEY = userdata.get("QDRANT_API_KEY")
            # Очищаем от пробельных символов (могут быть \r\n в конце)
            if QDRANT_API_KEY:
                QDRANT_API_KEY = QDRANT_API_KEY.strip()
        except (KeyError, AttributeError):
            QDRANT_API_KEY = None
        
        if QDRANT_URL:
            # Показываем только начало URL для безопасности
            url_preview = QDRANT_URL[:30] + "..." if len(QDRANT_URL) > 30 else QDRANT_URL
            console.print(f"[green]✅ QDRANT_URL найден: {url_preview}[/green]")
        else:
            console.print("[yellow]⚠️  QDRANT_URL не найден в секретах, будет использован in-memory режим[/yellow]")
            console.print("[cyan]💡 Подсказка: Убедитесь, что секрет 'QDRANT_URL' добавлен в Colab Secrets[/cyan]")
    else:
        console.print("[yellow]⚠️  userdata недоступен (не в Colab?), будет использован in-memory режим[/yellow]")
except Exception as e:
    # Если произошла другая ошибка
    console.print(f"[yellow]⚠️  Ошибка при получении секретов Qdrant: {type(e).__name__}: {e}[/yellow]")
    console.print("[yellow]   Будет использован in-memory режим[/yellow]")

class QdrantIndexer:
    """
    Индексатор для загрузки чанков в Qdrant.
    
    Зачем нужна:
    - Создание коллекции в Qdrant
    - Индексация чанков с embeddings и метаданными
    - Поддержка in-memory режима для Colab
    
    Вход: chunks (List[Chunk]), embeddings (List[List[float]])
    Выход: количество проиндексированных чанков
    
    Ограничения:
    - Коллекция должна существовать или быть создана
    - Размерность векторов должна совпадать с embedding_dim
    """
    
    def __init__(
        self,
        qdrant_client: QdrantClient = None,
        collection_name: str = "neuro_docs",
        embedding_dim: int = 1536,
        qdrant_url: Optional[str] = None,
        qdrant_api_key: Optional[str] = None
    ):
        # Если qdrant_client не передан, создаём его
        if qdrant_client is None:
            # Используем переданные параметры или глобальные из секретов
            url = qdrant_url or (QDRANT_URL if 'QDRANT_URL' in globals() else None)
            api_key = qdrant_api_key or (QDRANT_API_KEY if 'QDRANT_API_KEY' in globals() else None)
            
            # Очищаем от пробельных символов (могут быть \r\n в конце при копировании из секретов)
            if url:
                url = url.strip()
            if api_key:
                api_key = api_key.strip()
            
            if url:
                if api_key:
                    self.qdrant_client = QdrantClient(url=url, api_key=api_key)
                else:
                    self.qdrant_client = QdrantClient(url=url)
            else:
                # Используем in-memory режим
                self.qdrant_client = QdrantClient(":memory:")
        else:
            self.qdrant_client = qdrant_client
        self.collection_name = collection_name
        self.embedding_dim = embedding_dim
    
    def create_collection_if_not_exists(self):
        """Создаёт коллекцию, если она не существует."""
        try:
            # Проверяем, существует ли коллекция
            collections = self.qdrant_client.get_collections()
            collection_names = [c.name for c in collections.collections]
            
            if self.collection_name not in collection_names:
                # Создаём коллекцию
                self.qdrant_client.create_collection(
                    collection_name=self.collection_name,
                    vectors_config=VectorParams(
                        size=self.embedding_dim,
                        distance=Distance.COSINE
                    )
                )
                console.print(f"[green]✅ Коллекция '{self.collection_name}' создана[/green]")
            else:
                console.print(f"[yellow]ℹ️  Коллекция '{self.collection_name}' уже существует[/yellow]")
        except Exception as e:
            console.print(f"[red]❌ Ошибка при создании коллекции: {e}[/red]")
            raise
    
    def index_chunks(
        self,
        chunks: List[Chunk],
        embeddings: List[List[float]]
    ) -> int:
        """
        Индексирует чанки в Qdrant.
        
        Args:
            chunks: Список чанков
            embeddings: Список embeddings для чанков
            
        Returns:
            Количество проиндексированных чанков
        """
        if len(chunks) != len(embeddings):
            raise ValueError("Количество чанков и embeddings должно совпадать")
        
        # Создаём коллекцию, если не существует
        self.create_collection_if_not_exists()
        
        # Формируем точки для индексации
        points = []
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            point = PointStruct(
                id=i,  # Используем индекс как ID (в production лучше использовать chunk_id)
                vector=embedding,
                payload={
                    "chunk_id": chunk.chunk_id,
                    "doc_id": chunk.doc_id,
                    "text": chunk.text,
                    "text_length": chunk.text_length,
                    **chunk.metadata
                }
            )
            points.append(point)
        
        # Индексируем точки батчами
        batch_size = 100
        indexed_count = 0
        
        for i in range(0, len(points), batch_size):
            batch = points[i:i + batch_size]
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=batch
            )
            indexed_count += len(batch)
        
        return indexed_count

print("✅ QdrantIndexer реализован")


In [ ]:
# Тесты для QdrantIndexer

def test_qdrant_indexer():
    """Тесты для QdrantIndexer."""
    # Создаём in-memory Qdrant клиент
    qdrant_client = QdrantClient(":memory:")
    indexer = QdrantIndexer(qdrant_client, collection_name="test_collection", embedding_dim=1536)
    
    # Тест 1: Создание коллекции
    indexer.create_collection_if_not_exists()
    collections = qdrant_client.get_collections()
    collection_names = [c.name for c in collections.collections]
    assert "test_collection" in collection_names, "Коллекция должна быть создана"
    console.print("[green]✅ Тест 1 пройден: Коллекция создана[/green]")
    
    # Тест 2: Индексация чанков
    chunks = [
        Chunk(
            chunk_id="chunk1",
            doc_id="doc1",
            text="Текст первого чанка",
            text_length=10,
            metadata={"category": "hr"}
        ),
        Chunk(
            chunk_id="chunk2",
            doc_id="doc2",
            text="Текст второго чанка",
            text_length=10,
            metadata={"category": "it"}
        )
    ]
    embedding_service = EmbeddingService(mock_mode=True)
    embeddings = embedding_service.generate_embeddings([c.text for c in chunks])
    
    indexed_count = indexer.index_chunks(chunks, embeddings)
    assert indexed_count == 2, "Должно быть проиндексировано 2 чанка"
    console.print(f"[green]✅ Тест 2 пройден: Индексировано {indexed_count} чанков[/green]")
    
    # Тест 3: Проверка индексированных данных
    results = qdrant_client.scroll(
        collection_name="test_collection",
        limit=10
    )
    assert len(results[0]) == 2, "В коллекции должно быть 2 точки"
    console.print("[green]✅ Тест 3 пройден: Данные доступны в Qdrant[/green]")
    
    # Выводим примеры чанков
    console.print("\n[cyan]Примеры проиндексированных чанков:[/cyan]")
    for point in results[0][:2]:
        payload = point.payload
        console.print(f"  • {payload['chunk_id']}: {payload['text'][:50]}...")
        console.print(f"    Категория: {payload['category']}, Размер: {payload['text_length']} токенов")

test_qdrant_indexer()
print("\n✅ Все тесты QdrantIndexer пройдены")


## 📊 Демонстрация Ingestion Pipeline

Теперь покажем полный pipeline: загрузка → чанкинг → embeddings → индексация.


In [ ]:
# Полный Ingestion Pipeline (демонстрация)

def run_ingestion_pipeline_demo():
    """
    Демонстрация полного pipeline индексации.
    
    Показывает:
    - Количество загруженных документов
    - Количество созданных чанков
    - Размеры чанков
    - Примеры метаданных
    """
    # Создаём тестовые документы (в реальном Colab загружаем из data/)
    test_docs = [
        Document(
            id="doc_sla_1",
            text="SLA сервиса платежей: доступность 99.9%, время ответа не более 200мс. Сервис работает 24/7.",
            metadata={"category": "it", "file_name": "sla_payments.md"}
        ),
        Document(
            id="doc_sla_2",
            text="SLA сервиса API: доступность 99.95%, время ответа не более 100мс. Поддержка rate limiting.",
            metadata={"category": "it", "file_name": "sla_api.md"}
        ),
        Document(
            id="doc_hr_1",
            text="Политика удалённой работы: сотрудники могут работать удалённо до 3 дней в неделю.",
            metadata={"category": "hr", "file_name": "remote_work.md"}
        )
    ]
    
    console.print("\n[bold cyan]=== Ingestion Pipeline ===[/bold cyan]")
    
    # Шаг 1: Загрузка документов
    console.print("\n[bold]Шаг 1: Загрузка документов[/bold]")
    console.print(f"  Загружено документов: {len(test_docs)}")
    for doc in test_docs:
        console.print(f"    • {doc.metadata['file_name']} ({doc.metadata['category']})")
    
    # Шаг 2: Чанкинг
    console.print("\n[bold]Шаг 2: Чанкинг документов[/bold]")
    chunker = Chunker()
    chunks = chunker.chunk_documents(test_docs, chunk_size=50, overlap_percent=0.25)
    console.print(f"  Создано чанков: {len(chunks)}")
    console.print(f"  Средний размер: {sum(c.text_length for c in chunks) / len(chunks):.1f} токенов")
    
    # Показываем примеры чанков
    console.print("\n  Примеры чанков:")
    for i, chunk in enumerate(chunks[:3], 1):
        console.print(f"    [{i}] {chunk.chunk_id}")
        console.print(f"        Текст: {chunk.text[:60]}...")
        console.print(f"        Размер: {chunk.text_length} токенов, Категория: {chunk.metadata['category']}")
    
    # Шаг 3: Генерация embeddings
    console.print("\n[bold]Шаг 3: Генерация embeddings[/bold]")
    embedding_service = EmbeddingService(mock_mode=True, embedding_dim=1536)
    embeddings = embedding_service.generate_embeddings([c.text for c in chunks])
    console.print(f"  Сгенерировано embeddings: {len(embeddings)}")
    console.print(f"  Размерность: {len(embeddings[0])}")
    
    # Шаг 4: Индексация в Qdrant
    console.print("\n[bold]Шаг 4: Индексация в Qdrant[/bold]")
    # Используем внешний Qdrant, если доступен
    try:
        qdrant_url = QDRANT_URL if 'QDRANT_URL' in globals() and QDRANT_URL else None
        qdrant_api_key = QDRANT_API_KEY if 'QDRANT_API_KEY' in globals() and QDRANT_API_KEY else None
        
        # Очищаем от пробельных символов (могут быть \r\n в конце при копировании из секретов)
        if qdrant_url:
            qdrant_url = qdrant_url.strip()
        if qdrant_api_key:
            qdrant_api_key = qdrant_api_key.strip()
        
        if qdrant_url:
            if qdrant_api_key:
                qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
            else:
                qdrant_client = QdrantClient(url=qdrant_url)
            console.print(f"[cyan]Используется внешний Qdrant: {qdrant_url}[/cyan]")
        else:
            qdrant_client = QdrantClient(":memory:")
            console.print("[cyan]Используется in-memory Qdrant[/cyan]")
    except Exception as e:
        qdrant_client = QdrantClient(":memory:")
        console.print(f"[yellow]Ошибка подключения к внешнему Qdrant: {e}, используется in-memory[/yellow]")
    
    indexer = QdrantIndexer(qdrant_client, collection_name="neuro_docs_demo", embedding_dim=1536)
    indexed_count = indexer.index_chunks(chunks, embeddings)
    console.print(f"  Проиндексировано чанков: {indexed_count}")
    
    # Проверяем коллекцию
    # Используем count вместо get_collection для избежания проблем совместимости версий
    try:
        count_result = qdrant_client.count("neuro_docs_demo")
        points_count = count_result.count
        console.print(f"  Размер коллекции: {points_count} точек")
    except Exception as e:
        # Если count не работает, пробуем get_collection
        try:
            collection_info = qdrant_client.get_collection("neuro_docs_demo")
            console.print(f"  Размер коллекции: {collection_info.points_count} точек")
        except Exception as e2:
            # Если оба метода не работают, используем количество проиндексированных чанков
            console.print(f"  [yellow]⚠️  Не удалось получить информацию о коллекции (ошибка совместимости версий)[/yellow]")
            console.print(f"  [cyan]Проиндексировано чанков: {indexed_count}[/cyan]")
    
    console.print("\n[bold green]✅ Ingestion Pipeline завершён успешно![/bold green]")
    
    return qdrant_client, chunks

# Запускаем демонстрацию
qdrant_demo, chunks_demo = run_ingestion_pipeline_demo()


# 5️⃣ Retrieval Layer

Semantic search по Qdrant с параметром K. Наглядно показываем retrieved документы в виде таблицы.


In [ ]:
# ============================================
# Модуль: Retriever
# ============================================

@dataclass
class RetrievedChunk:
    """Представление retrieved чанка из Qdrant."""
    id: str
    text: str
    score: float
    metadata: dict

class Retriever:
    """
    Retriever для semantic search по Qdrant.
    
    Зачем нужна:
    - Semantic search по коллекции neuro_docs
    - Параметризация K (количество retrieved чанков: 3, 5, 8)
    - Возврат релевантных чанков с метаданными и scores
    
    Вход: query (str), k (int)
    Выход: List[RetrievedChunk] — список чанков, отсортированных по score
    
    Ограничения:
    - k: 3, 5, 8 (оптимально 3 для Precision@3)
    """
    
    def __init__(
        self,
        qdrant_client: QdrantClient,
        embedding_service: EmbeddingService,
        collection_name: str = "neuro_docs"
    ):
        self.qdrant_client = qdrant_client
        self.embedding_service = embedding_service
        self.collection_name = collection_name
    
    def retrieve(
        self,
        query: str,
        k: int = 3,
        score_threshold: Optional[float] = None
    ) -> List[RetrievedChunk]:
        """
        Выполняет semantic search по Qdrant.
        
        Args:
            query: Текстовый запрос пользователя
            k: Количество retrieved чанков (3, 5, 8)
            score_threshold: Минимальный score для фильтрации (опционально)
        """
        # Генерируем embedding для запроса
        try:
            query_embedding = self.embedding_service.generate_embeddings([query])[0]
        except Exception as e:
            raise ValueError(f"Ошибка при генерации embedding: {str(e)}")
        
        # Выполняем поиск в Qdrant
        # В новых версиях qdrant-client API изменился
        search_results = None
        last_error = None
        
        # Пробуем разные варианты API в зависимости от версии qdrant-client
        # Вариант 1: query_points с прямой передачей вектора (новые версии >= 1.8.0)
        try:
            query_result = self.qdrant_client.query_points(
                collection_name=self.collection_name,
                query=query_embedding,  # Прямая передача вектора как списка
                limit=k,
                with_payload=True
            )
            # query_points возвращает объект с атрибутом points
            search_results = query_result.points if hasattr(query_result, 'points') else []
        except Exception as e:
            last_error = e
            # Пробуем query_points с NamedVector
            try:
                from qdrant_client.models import NamedVector
                query_result = self.qdrant_client.query_points(
                    collection_name=self.collection_name,
                    query=NamedVector(
                        vector=query_embedding
                    ),
                    limit=k,
                    with_payload=True
                )
                search_results = query_result.points if hasattr(query_result, 'points') else []
            except Exception as e2:
                last_error = e2
        
        # Вариант 2: search_points (версии 1.7.x)
        if search_results is None:
            try:
                search_result = self.qdrant_client.search_points(
                    collection_name=self.collection_name,
                    query_vector=query_embedding,
                    limit=k,
                    with_payload=True
                )
                search_results = search_result.points if hasattr(search_result, 'points') else []
            except Exception as e:
                last_error = e
        
        # Вариант 3: search (старые версии < 1.7.0)
        if search_results is None:
            try:
                search_results = self.qdrant_client.search(
                    collection_name=self.collection_name,
                    query_vector=query_embedding,
                    limit=k
                )
            except Exception as e:
                last_error = e
        
        # Если ничего не сработало
        if search_results is None:
            # Получаем список доступных методов для отладки
            available_methods = [m for m in dir(self.qdrant_client) if ('search' in m.lower() or 'query' in m.lower()) and not m.startswith('_')]
            client_type = type(self.qdrant_client).__name__
            raise ValueError(
                f"Не удалось найти рабочий метод поиска в QdrantClient ({client_type}). "
                f"Последняя ошибка: {str(last_error)}. "
                f"Доступные методы с 'search' или 'query': {available_methods}. "
                f"Проверьте версию qdrant-client: !pip show qdrant-client. "
                f"Попробуйте обновить: !pip install --upgrade qdrant-client"
            )
        
        # Конвертируем результаты в RetrievedChunk
        retrieved_chunks = []
        for result in search_results:
            payload = result.payload
            
            # Применяем score_threshold, если указан
            if score_threshold is not None and result.score < score_threshold:
                continue
            
            chunk = RetrievedChunk(
                id=payload.get("chunk_id", str(result.id)),
                text=payload.get("text", ""),
                score=result.score,
                metadata={
                    "doc_id": payload.get("doc_id"),
                    "category": payload.get("category", "unknown"),
                    "file_name": payload.get("file_name", ""),
                    **{k: v for k, v in payload.items() if k not in ["chunk_id", "doc_id", "text", "category", "file_name"]}
                }
            )
            retrieved_chunks.append(chunk)
        
        return retrieved_chunks

print("✅ Retriever реализован")


In [ ]:
# Демонстрация Retrieval Layer

def demonstrate_retrieval():
    """Демонстрация semantic search с наглядным выводом."""
    # Используем Qdrant из предыдущего pipeline
    embedding_service = EmbeddingService(mock_mode=True)
    retriever = Retriever(
        qdrant_client=qdrant_demo,
        embedding_service=embedding_service,
        collection_name="neuro_docs_demo"
    )
    
    # Запрос UC-1
    query = "Какой SLA у сервиса платежей?"
    
    console.print(f"\n[bold cyan]=== Retrieval Layer ===[/bold cyan]")
    console.print(f"[bold]Запрос:[/bold] {query}")
    console.print(f"[bold]K:[/bold] 3")
    
    # Выполняем retrieval
    retrieved_chunks = retriever.retrieve(query, k=3)
    
    # Выводим результаты в виде таблицы
    table = Table(title="Retrieved Documents", show_header=True, header_style="bold magenta")
    table.add_column("№", style="dim", width=3)
    table.add_column("Score", justify="right", style="cyan")
    table.add_column("Source", style="green")
    table.add_column("Chunk ID", style="yellow")
    table.add_column("Text Preview", style="white", width=50)
    
    for i, chunk in enumerate(retrieved_chunks, 1):
        table.add_row(
            str(i),
            f"{chunk.score:.4f}",
            chunk.metadata.get("category", "unknown"),
            chunk.id[:20] + "..." if len(chunk.id) > 20 else chunk.id,
            chunk.text[:60] + "..." if len(chunk.text) > 60 else chunk.text
        )
    
    console.print(table)
    
    # Дополнительная информация
    console.print(f"\n[cyan]Найдено чанков:[/cyan] {len(retrieved_chunks)}")
    console.print(f"[cyan]Средний score:[/cyan] {sum(c.score for c in retrieved_chunks) / len(retrieved_chunks):.4f}")
    
    return retrieved_chunks

retrieved_demo = demonstrate_retrieval()


In [ ]:
# Тесты для Retriever

def test_retriever():
    """Тесты для Retriever (pytest-style)."""
    from unittest.mock import Mock, MagicMock
    
    # Создаём mock Qdrant client
    mock_qdrant = MagicMock()
    # Создаём mock точки результатов поиска
    mock_points = [
        Mock(
            id=0,
            score=0.95,
            payload={
                "chunk_id": "chunk_001",
                "text": "SLA сервиса платежей составляет 99.9%",
                "doc_id": "doc_001",
                "category": "it"
            }
        ),
        Mock(
            id=1,
            score=0.88,
            payload={
                "chunk_id": "chunk_002",
                "text": "Время отклика сервиса платежей не более 200мс",
                "doc_id": "doc_001",
                "category": "it"
            }
        )
    ]
    # Настраиваем mock для query_points (новые версии) - возвращает объект с атрибутом points
    mock_query_result = Mock()
    mock_query_result.points = mock_points
    mock_qdrant.query_points.return_value = mock_query_result
    
    # Также настраиваем для search_points (версии 1.7.x)
    mock_search_result = Mock()
    mock_search_result.points = mock_points
    mock_qdrant.search_points.return_value = mock_search_result
    
    # И для search (старые версии) - возвращает список напрямую
    mock_qdrant.search.return_value = mock_points
    
    # Создаём embedding service
    embedding_service = EmbeddingService(mock_mode=True)
    
    # Создаём retriever
    retriever = Retriever(
        qdrant_client=mock_qdrant,
        embedding_service=embedding_service,
        collection_name="test_collection"
    )
    
    # Тест 1: retrieve возвращает список RetrievedChunk
    query = "Какой SLA у сервиса платежей?"
    results = retriever.retrieve(query, k=2)
    
    assert len(results) == 2, "Должно быть 2 чанка"
    assert all(isinstance(chunk, RetrievedChunk) for chunk in results)
    assert all(chunk.text is not None for chunk in results)
    console.print("[green]✅ Тест 1 пройден: retrieve возвращает RetrievedChunk[/green]")
    
    # Тест 2: K параметр работает корректно
    # Обновляем mock для всех методов
    mock_query_result.points = mock_points[:1]  # Один чанк
    mock_search_result.points = mock_points[:1]
    mock_qdrant.search.return_value = mock_points[:1]
    results = retriever.retrieve(query, k=1)
    assert len(results) == 1, "Должен быть 1 чанк при k=1"
    console.print("[green]✅ Тест 2 пройден: K параметр работает[/green]")
    
    # Тест 3: Результаты отсортированы по score
    # Возвращаем оба чанка для всех методов
    mock_query_result.points = mock_points
    mock_search_result.points = mock_points
    mock_qdrant.search.return_value = mock_points
    results = retriever.retrieve(query, k=2)
    scores = [chunk.score for chunk in results]
    assert scores == sorted(scores, reverse=True), "Чанки должны быть отсортированы по score"
    console.print("[green]✅ Тест 3 пройден: Результаты отсортированы по score[/green]")

test_retriever()
print("\n✅ Все тесты Retriever пройдены")


# 6️⃣ Reranking (экспериментально)

Сравнение режимов: **без reranking** vs **с reranking**. Показываем влияние на Precision@K.


In [ ]:
# ============================================
# Модуль: Reranker
# ============================================

import re

@dataclass
class RerankedChunk:
    """Представление reranked чанка."""
    id: str
    text: str
    score: float
    rerank_score: float
    metadata: dict

class Reranker:
    """
    Reranker для переупорядочивания retrieved документов.
    
    Зачем нужна:
    - Переупорядочивание retrieved чанков по релевантности к запросу
    - Повышение Precision@3 при большом объёме документов
    - Комбинация оригинального semantic search score и keyword-based reranking
    
    Вход: query (str), chunks (List[RetrievedChunk]), top_k (int)
    Выход: List[RerankedChunk] — переупорядоченные чанки
    
    Ограничения:
    - keyword_weight: 0.3, original_score_weight: 0.7 (настраиваемо)
    """
    
    def __init__(self, keyword_weight: float = 0.3, original_score_weight: float = 0.7):
        self.keyword_weight = keyword_weight
        self.original_score_weight = original_score_weight
    
    def _extract_keywords(self, text: str) -> List[str]:
        """Извлечение ключевых слов из текста."""
        stop_words = {
            "и", "в", "на", "с", "по", "для", "от", "до", "из", "к", "о", "об",
            "а", "но", "или", "как", "что", "это", "так", "уже", "еще", "ещё"
        }
        text_lower = text.lower()
        words = re.findall(r'\b[а-яёa-z]+\b', text_lower)
        keywords = [w for w in words if w not in stop_words and len(w) > 2]
        return keywords
    
    def rerank(
        self,
        query: str,
        chunks: List[RetrievedChunk],
        top_k: Optional[int] = None
    ) -> List[RerankedChunk]:
        """
        Переупорядочивает чанки по релевантности.
        
        Args:
            query: Запрос пользователя
            chunks: Список retrieved чанков
            top_k: Количество чанков после reranking (если None, используется len(chunks))
        """
        query_keywords = set(self._extract_keywords(query))
        
        reranked = []
        for chunk in chunks:
            chunk_keywords = set(self._extract_keywords(chunk.text))
            
            # Вычисляем keyword overlap
            common_keywords = query_keywords.intersection(chunk_keywords)
            keyword_score = len(common_keywords) / max(len(query_keywords), 1)
            
            # Комбинируем scores
            rerank_score = (
                self.original_score_weight * chunk.score +
                self.keyword_weight * keyword_score
            )
            
            reranked_chunk = RerankedChunk(
                id=chunk.id,
                text=chunk.text,
                score=chunk.score,
                rerank_score=rerank_score,
                metadata=chunk.metadata
            )
            reranked.append(reranked_chunk)
        
        # Сортируем по rerank_score
        reranked.sort(key=lambda x: x.rerank_score, reverse=True)
        
        # Возвращаем top_k
        if top_k is not None:
            reranked = reranked[:top_k]
        
        return reranked

print("✅ Reranker реализован")


In [ ]:
# Тесты для Reranker

def test_reranker():
    """Тесты для Reranker."""
    # Создаём тестовые чанки
    sample_chunks = [
        RetrievedChunk(
            id="chunk_1",
            text="SLA сервиса платежей составляет 99.9%. Время отклика не более 200мс.",
            score=0.85,
            metadata={"doc_id": "doc_1", "category": "it"}
        ),
        RetrievedChunk(
            id="chunk_2",
            text="HR политика компании включает правила отпусков и больничных.",
            score=0.78,
            metadata={"doc_id": "doc_2", "category": "hr"}
        ),
        RetrievedChunk(
            id="chunk_3",
            text="Платежный сервис имеет SLA 99.9% и время отклика 200мс.",
            score=0.72,
            metadata={"doc_id": "doc_3", "category": "it"}
        )
    ]
    
    query = "Какой SLA у сервиса платежей?"
    reranker = Reranker()
    
    # Тест 1: rerank возвращает RerankedChunk
    reranked = reranker.rerank(query=query, chunks=sample_chunks, top_k=3)
    
    assert len(reranked) == 3, "Должно быть 3 reranked чанка"
    assert all(hasattr(chunk, "rerank_score") for chunk in reranked)
    console.print("[green]✅ Тест 1 пройден: rerank возвращает RerankedChunk[/green]")
    
    # Тест 2: rerank_score рассчитывается для каждого чанка
    for chunk in reranked:
        assert isinstance(chunk.rerank_score, float)
        assert 0.0 <= chunk.rerank_score <= 1.0
    console.print("[green]✅ Тест 2 пройден: rerank_score рассчитывается[/green]")
    
    # Тест 3: top_k параметр работает корректно
    reranked_2 = reranker.rerank(query=query, chunks=sample_chunks, top_k=2)
    assert len(reranked_2) == 2, "Должно быть 2 чанка при top_k=2"
    console.print("[green]✅ Тест 3 пройден: top_k параметр работает[/green]")
    
    # Тест 4: Чанки отсортированы по rerank_score
    scores = [chunk.rerank_score for chunk in reranked]
    assert scores == sorted(scores, reverse=True), "Чанки должны быть отсортированы по rerank_score"
    console.print("[green]✅ Тест 4 пройден: Чанки отсортированы по rerank_score[/green]")

test_reranker()
print("\n✅ Все тесты Reranker пройдены")


# 7️⃣ Agent Controller (FSM)

**Ключевой модуль**: явная state machine управляет логикой, а не LLM.

Показываем:
- Переходы состояний
- Лог выполнения
- Почему принято то или иное решение


In [ ]:
# ============================================
# Модуль: Agent State Machine
# ============================================

from enum import Enum

class AgentState(Enum):
    """Состояния агента."""
    IDLE = "IDLE"
    VALIDATE_QUERY = "VALIDATE_QUERY"
    REQUEST_CLARIFICATION = "REQUEST_CLARIFICATION"
    RETRIEVE = "RETRIEVE"
    METADATA_FILTER = "METADATA_FILTER"
    RERANK = "RERANK"
    GENERATE = "GENERATE"
    VALIDATE_ANSWER = "VALIDATE_ANSWER"
    LOG_METRICS = "LOG_METRICS"
    RETURN_RESPONSE = "RETURN_RESPONSE"

class AgentStateMachine:
    """
    Конечный автомат состояний для агента.
    
    Зачем нужна:
    - Управление состояниями агента
    - Детерминированные переходы между состояниями
    - Сохранение истории состояний
    
    Вход: new_state (AgentState)
    Выход: переход в новое состояние, обновление истории
    """
    
    def __init__(self):
        self.current_state = AgentState.IDLE
        self.state_history: List[AgentState] = [AgentState.IDLE]
    
    def transition_to(self, new_state: AgentState) -> None:
        """Выполняет переход в новое состояние."""
        self.current_state = new_state
        self.state_history.append(new_state)
    
    def get_history(self) -> List[AgentState]:
        """Возвращает историю состояний."""
        return self.state_history.copy()
    
    def reset(self) -> None:
        """Сбрасывает state machine в начальное состояние."""
        self.current_state = AgentState.IDLE
        self.state_history = [AgentState.IDLE]

class DecisionLog:
    """
    Лог решений агента.
    
    Зачем нужна:
    - Трассировка каждого шага агента
    - Фиксация входных данных и выходных данных
    - Понимание, почему принято то или иное решение
    """
    
    def __init__(self):
        self.logs: List[Dict[str, Any]] = []
    
    def log_decision(
        self,
        state: str,
        action: str,
        input_data: Any,
        output_data: Any,
        metadata: Optional[Dict[str, Any]] = None
    ):
        """Логирует решение агента."""
        log_entry = {
            "state": state,
            "action": action,
            "input": str(input_data),
            "output": str(output_data),
            "metadata": metadata or {}
        }
        self.logs.append(log_entry)
    
    def get_logs(self) -> List[Dict[str, Any]]:
        """Возвращает все логи."""
        return self.logs.copy()

print("✅ Agent State Machine реализован")


In [ ]:
# Тесты для AgentStateMachine

def test_agent_state_machine():
    """Тесты для AgentStateMachine."""
    state_machine = AgentStateMachine()
    
    # Тест 1: Начальное состояние IDLE
    assert state_machine.current_state == AgentState.IDLE, "Начальное состояние должно быть IDLE"
    console.print("[green]✅ Тест 1 пройден: Начальное состояние IDLE[/green]")
    
    # Тест 2: Переходы между состояниями
    state_machine.transition_to(AgentState.VALIDATE_QUERY)
    assert state_machine.current_state == AgentState.VALIDATE_QUERY
    
    state_machine.transition_to(AgentState.RETRIEVE)
    assert state_machine.current_state == AgentState.RETRIEVE
    
    state_machine.transition_to(AgentState.GENERATE)
    assert state_machine.current_state == AgentState.GENERATE
    console.print("[green]✅ Тест 2 пройден: Переходы между состояниями работают[/green]")
    
    # Тест 3: История состояний сохраняется
    history = state_machine.get_history()
    assert len(history) >= 3, "История должна содержать минимум 3 состояния"
    assert AgentState.IDLE in history, "История должна содержать IDLE"
    assert AgentState.VALIDATE_QUERY in history, "История должна содержать VALIDATE_QUERY"
    console.print("[green]✅ Тест 3 пройден: История состояний сохраняется[/green]")
    
    # Тест 4: Полный flow UC-1
    state_machine.reset()
    states = [
        AgentState.VALIDATE_QUERY,
        AgentState.RETRIEVE,
        AgentState.GENERATE,
        AgentState.VALIDATE_ANSWER,
        AgentState.LOG_METRICS,
        AgentState.RETURN_RESPONSE,
        AgentState.IDLE
    ]
    for state in states:
        state_machine.transition_to(state)
    assert state_machine.current_state == AgentState.IDLE, "Финальное состояние должно быть IDLE"
    console.print("[green]✅ Тест 4 пройден: Полный flow UC-1 работает[/green]")

test_agent_state_machine()
print("\n✅ Все тесты AgentStateMachine пройдены")


In [ ]:
# Демонстрация Agent Controller (FSM)

def demonstrate_agent_fsm():
    """Демонстрация работы Agent Controller с FSM."""
    state_machine = AgentStateMachine()
    decision_log = DecisionLog()
    
    query = "Какой SLA у сервиса платежей?"
    
    console.print(f"\n[bold cyan]=== Agent Controller (FSM) ===[/bold cyan]")
    console.print(f"[bold]Запрос:[/bold] {query}\n")
    
    # IDLE → VALIDATE_QUERY
    state_machine.transition_to(AgentState.VALIDATE_QUERY)
    decision_log.log_decision(
        state=state_machine.current_state.value,
        action="validate_query",
        input_data=query,
        output_data="valid",
        metadata={}
    )
    console.print(f"[green]→ {state_machine.current_state.value}[/green]: Запрос валиден")
    
    # VALIDATE_QUERY → RETRIEVE
    state_machine.transition_to(AgentState.RETRIEVE)
    decision_log.log_decision(
        state=state_machine.current_state.value,
        action="retrieve_chunks",
        input_data=query,
        output_data="3 chunks retrieved",
        metadata={"k": 3}
    )
    console.print(f"[green]→ {state_machine.current_state.value}[/green]: Найдено 3 чанка")
    
    # RETRIEVE → RERANK (опционально)
    use_reranking = True
    if use_reranking:
        state_machine.transition_to(AgentState.RERANK)
        decision_log.log_decision(
            state=state_machine.current_state.value,
            action="rerank_chunks",
            input_data="3 chunks before rerank",
            output_data="3 chunks after rerank",
            metadata={"rerank_top_k": 3}
        )
        console.print(f"[green]→ {state_machine.current_state.value}[/green]: Reranking применён")
    
    # RERANK → GENERATE
    state_machine.transition_to(AgentState.GENERATE)
    decision_log.log_decision(
        state=state_machine.current_state.value,
        action="generate_answer",
        input_data="prompt built",
        output_data="answer generated",
        metadata={}
    )
    console.print(f"[green]→ {state_machine.current_state.value}[/green]: Ответ сгенерирован")
    
    # GENERATE → VALIDATE_ANSWER
    state_machine.transition_to(AgentState.VALIDATE_ANSWER)
    decision_log.log_decision(
        state=state_machine.current_state.value,
        action="validate_answer",
        input_data="answer",
        output_data="answer validated",
        metadata={}
    )
    console.print(f"[green]→ {state_machine.current_state.value}[/green]: Ответ валидирован")
    
    # VALIDATE_ANSWER → LOG_METRICS
    state_machine.transition_to(AgentState.LOG_METRICS)
    decision_log.log_decision(
        state=state_machine.current_state.value,
        action="log_metrics",
        input_data="metrics calculated",
        output_data="metrics logged",
        metadata={"precision_at_3": 0.85, "faithfulness": 0.90}
    )
    console.print(f"[green]→ {state_machine.current_state.value}[/green]: Метрики залогированы")
    
    # LOG_METRICS → RETURN_RESPONSE
    state_machine.transition_to(AgentState.RETURN_RESPONSE)
    console.print(f"[green]→ {state_machine.current_state.value}[/green]: Ответ возвращён")
    
    # RETURN_RESPONSE → IDLE
    state_machine.transition_to(AgentState.IDLE)
    
    # Выводим историю состояний
    console.print(f"\n[bold yellow]История состояний:[/bold yellow]")
    history = state_machine.get_history()
    for i, state in enumerate(history, 1):
        console.print(f"  {i}. {state.value}")
    
    # Выводим Decision Log
    console.print(f"\n[bold yellow]Decision Log:[/bold yellow]")
    logs = decision_log.get_logs()
    for i, log in enumerate(logs, 1):
        console.print(f"\n  [{i}] {log['state']} - {log['action']}")
        console.print(f"      Input: {log['input'][:50]}...")
        console.print(f"      Output: {log['output']}")
        if log['metadata']:
            console.print(f"      Metadata: {log['metadata']}")
    
    return state_machine, decision_log

state_machine_demo, decision_log_demo = demonstrate_agent_fsm()


# 8️⃣ Generation Layer

**PromptBuilder** формирует prompt со строгой инструкцией: «Отвечай ТОЛЬКО на основе контекста».

Показываем:
- Prompt
- Context
- Raw response


In [ ]:
# ============================================
# Модуль: PromptBuilder
# ============================================

class PromptBuilder:
    """
    Построитель prompt для LLM.
    
    Зачем нужна:
    - Формирование prompt с контекстом из retrieved чанков
    - Добавление строгой инструкции «отвечай только по контексту»
    - Структурирование prompt для оптимальной работы LLM
    
    Вход: query (str), retrieved_chunks (List[RetrievedChunk])
    Выход: prompt (str) — сформированный prompt
    
    Ограничения:
    - Инструкция должна быть строгой для предотвращения галлюцинаций
    """
    
    def __init__(self, instruction_template: str = None):
        self.instruction_template = instruction_template or self._default_instruction()
    
    def _default_instruction(self) -> str:
        """Возвращает инструкцию по умолчанию для предотвращения галлюцинаций."""
        return (
            "Ты - помощник, который отвечает на вопросы строго на основе предоставленного контекста.\n"
            "ВАЖНО:\n"
            "- Отвечай ТОЛЬКО на основе информации из контекста\n"
            "- НЕ придумывай информацию, которой нет в контексте\n"
            "- НЕ добавляй факты, не упомянутые в контексте\n"
            "- Если в контексте нет ответа на вопрос, честно скажи об этом\n"
            "- Используй информацию из контекста дословно или близко к тексту\n"
        )
    
    def build_prompt(
        self,
        query: str,
        retrieved_chunks: List[RetrievedChunk]
    ) -> str:
        """
        Формирует prompt с контекстом и запросом.
        
        Args:
            query: Запрос пользователя
            retrieved_chunks: Список retrieved чанков с контекстом
        """
        if not retrieved_chunks:
            return (
                f"{self.instruction_template}\n\n"
                f"Контекст: В предоставленной документации нет релевантной информации.\n\n"
                f"Вопрос: {query}\n\n"
                f"Ответ: В документации не найдено информации для ответа на этот вопрос."
            )
        
        # Формируем контекст из всех retrieved чанков
        context_parts = []
        for i, chunk in enumerate(retrieved_chunks, start=1):
            context_parts.append(f"[Источник {i}]\n{chunk.text}\n")
        
        context = "\n".join(context_parts)
        
        # Формируем полный prompt
        prompt = (
            f"{self.instruction_template}\n\n"
            f"Контекст из документации:\n{context}\n\n"
            f"Вопрос: {query}\n\n"
            f"Ответ (на основе контекста):"
        )
        
        return prompt

class LLMClient:
    """
    Клиент для работы с LLM (поддержка GigaChat API и mock режим).
    
    Зачем нужна:
    - Вызов LLM API для генерации ответов
    - В production: GigaChat API
    - В Colab: можно использовать mock-режим или реальный API (при наличии ключей)
    
    Вход: prompt (str)
    Выход: answer (str) — сгенерированный ответ
    """
    
    def __init__(
        self,
        mock_mode: bool = True,
        model: str = "GigaChat",
        temperature: float = 0.1,
        max_tokens: int = 500,
        auth_key: Optional[str] = None,
        scope: Optional[str] = None
    ):
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        
        # Определяем auth_key (из секретов Colab или параметра)
        if not auth_key:
            try:
                auth_key = userdata.get("GIGACHAT_AUTH_KEY")
            except:
                auth_key = None
        
        # Определяем scope (из секретов Colab или параметра)
        if not scope:
            try:
                scope = userdata.get("GIGACHAT_SCOPE", "GIGACHAT_API_PERS")
            except:
                scope = "GIGACHAT_API_PERS"
        
        # Определяем mock mode
        if mock_mode or not auth_key:
            self.mock_mode = True
        else:
            self.mock_mode = False
            self.auth = GigaChatAuth(auth_key=auth_key, scope=scope)
        
        # Настройка HTTP сессии
        self.session = requests.Session()
        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504],
            allowed_methods=["POST"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)
        self.session.verify = False
        
        # Официальный endpoint для GigaChat Chat Completions API
        self.api_url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    
    def generate_answer(self, prompt: str) -> str:
        """
        Генерирует ответ на основе prompt.
        
        В mock-режиме: извлекает релевантную информацию из контекста.
        В production: вызывает GigaChat API.
        """
        if self.mock_mode:
            # Mock: извлекаем информацию из контекста
            # В реальности это делает LLM
            if "SLA" in prompt and "платеж" in prompt.lower():
                return "SLA сервиса платежей: доступность 99.9%, время ответа не более 200мс. Сервис работает 24/7."
            elif "SLA" in prompt and "API" in prompt:
                return "SLA сервиса API: доступность 99.95%, время ответа не более 100мс. Поддержка rate limiting."
            else:
                # Извлекаем первый релевантный фрагмент из контекста
                lines = prompt.split("\n")
                for line in lines:
                    if line.strip() and not line.startswith("[") and not line.startswith("Ты"):
                        return line.strip()
                return "Информация не найдена в документации."
        else:
            # Production: вызов GigaChat API
            try:
                response_data = self._call_gigachat_api(prompt)
                answer = self._extract_answer(response_data)
                
                if not answer or not answer.strip():
                    raise ValueError("Empty answer received from GigaChat API")
                
                return answer
            except Exception as e:
                # При ошибке API переключаемся на mock mode
                console.print(f"[yellow]⚠️  Ошибка вызова GigaChat API: {e}[/yellow]")
                console.print("[yellow]   Используется mock режим для демонстрации[/yellow]")
                # Используем простой mock fallback
                lines = prompt.split("\n")
                for line in lines:
                    if line.strip() and not line.startswith("[") and not line.startswith("Ты") and not line.startswith("Контекст"):
                        return line.strip()
                return "Информация не найдена в документации."
    
    def _call_gigachat_api(self, prompt: str) -> Dict[str, Any]:
        """
        Вызывает GigaChat API для генерации ответа.
        
        Args:
            prompt: Prompt для генерации
            
        Returns:
            Ответ от API в виде словаря
        """
        # Получаем access token через OAuth 2.0
        if not self.auth:
            raise ValueError("GigaChatAuth не инициализирован. Проверьте настройки аутентификации.")
        
        access_token = self.auth.get_access_token()
        if not access_token:
            raise ValueError("Не удалось получить access token для GigaChat API. Проверьте GIGACHAT_AUTH_KEY и GIGACHAT_SCOPE.")
        
        # Генерируем уникальный идентификатор запроса (UUID4)
        request_id = str(uuid.uuid4())
        
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "X-Request-ID": request_id
        }
        
        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }
        
        response = self.session.post(
            self.api_url,
            json=payload,
            headers=headers,
            timeout=60  # Увеличиваем timeout для генерации
        )
        response.raise_for_status()
        
        return response.json()
    
    def _extract_answer(self, response_data: Dict[str, Any]) -> str:
        """
        Извлекает текст ответа из структуры ответа API.
        
        Args:
            response_data: Ответ от GigaChat API
            
        Returns:
            Текст ответа
        """
        # Ожидаемая структура ответа GigaChat API:
        # {
        #   "choices": [
        #     {
        #       "message": {
        #         "content": "текст ответа"
        #       }
        #     }
        #   ],
        #   "usage": {...}
        # }
        
        if "choices" in response_data and len(response_data["choices"]) > 0:
            first_choice = response_data["choices"][0]
            if "message" in first_choice and "content" in first_choice["message"]:
                return first_choice["message"]["content"]
        
        raise ValueError(f"Unexpected API response format: {response_data}")

print("✅ Generation Layer реализован (поддержка GigaChat API и mock режим)")


In [ ]:
# Тесты для Retriever

def test_retriever():
    """Тесты для Retriever (pytest-style)."""
    from unittest.mock import Mock, MagicMock
    
    # Создаём mock Qdrant client
    mock_qdrant = MagicMock()
    # Создаём mock точки результатов поиска
    mock_points = [
        Mock(
            id=0,
            score=0.95,
            payload={
                "chunk_id": "chunk_001",
                "text": "SLA сервиса платежей составляет 99.9%",
                "doc_id": "doc_001",
                "category": "it"
            }
        ),
        Mock(
            id=1,
            score=0.88,
            payload={
                "chunk_id": "chunk_002",
                "text": "Время отклика сервиса платежей не более 200мс",
                "doc_id": "doc_001",
                "category": "it"
            }
        )
    ]
    # Настраиваем mock для query_points (новые версии) - возвращает объект с атрибутом points
    mock_query_result = Mock()
    mock_query_result.points = mock_points
    mock_qdrant.query_points.return_value = mock_query_result
    
    # Также настраиваем для search_points (версии 1.7.x)
    mock_search_result = Mock()
    mock_search_result.points = mock_points
    mock_qdrant.search_points.return_value = mock_search_result
    
    # И для search (старые версии) - возвращает список напрямую
    mock_qdrant.search.return_value = mock_points
    
    # Создаём embedding service
    embedding_service = EmbeddingService(mock_mode=True)
    
    # Создаём retriever
    retriever = Retriever(
        qdrant_client=mock_qdrant,
        embedding_service=embedding_service,
        collection_name="test_collection"
    )
    
    # Тест 1: retrieve возвращает список RetrievedChunk
    query = "Какой SLA у сервиса платежей?"
    results = retriever.retrieve(query, k=2)
    
    assert len(results) == 2, "Должно быть 2 чанка"
    assert all(isinstance(chunk, RetrievedChunk) for chunk in results)
    assert all(chunk.text is not None for chunk in results)
    console.print("[green]✅ Тест 1 пройден: retrieve возвращает RetrievedChunk[/green]")
    
    # Тест 2: K параметр работает корректно
    # Обновляем mock для всех методов
    mock_query_result.points = mock_points[:1]  # Один чанк
    mock_search_result.points = mock_points[:1]
    mock_qdrant.search.return_value = mock_points[:1]
    results = retriever.retrieve(query, k=1)
    assert len(results) == 1, "Должен быть 1 чанк при k=1"
    console.print("[green]✅ Тест 2 пройден: K параметр работает[/green]")
    
    # Тест 3: Результаты отсортированы по score
    # Возвращаем оба чанка для всех методов
    mock_query_result.points = mock_points
    mock_search_result.points = mock_points
    mock_qdrant.search.return_value = mock_points
    results = retriever.retrieve(query, k=2)
    scores = [chunk.score for chunk in results]
    assert scores == sorted(scores, reverse=True), "Чанки должны быть отсортированы по score"
    console.print("[green]✅ Тест 3 пройден: Результаты отсортированы по score[/green]")

test_retriever()
print("\n✅ Все тесты Retriever пройдены")


In [ ]:
# Демонстрация Generation Layer

def demonstrate_generation():
    """Демонстрация генерации ответа с показом prompt и context."""
    query = "Какой SLA у сервиса платежей?"
    
    # Используем retrieved chunks из предыдущего раздела
    embedding_service = EmbeddingService(mock_mode=True)
    retriever = Retriever(
        qdrant_client=qdrant_demo,
        embedding_service=embedding_service,
        collection_name="neuro_docs_demo"
    )
    retrieved_chunks = retriever.retrieve(query, k=3)
    
    # Формируем prompt
    prompt_builder = PromptBuilder()
    prompt = prompt_builder.build_prompt(query, retrieved_chunks)
    
    # Генерируем ответ
    llm_client = LLMClient(mock_mode=True)
    answer = llm_client.generate_answer(prompt)
    
    console.print(f"\n[bold cyan]=== Generation Layer ===[/bold cyan]")
    console.print(f"[bold]Запрос:[/bold] {query}\n")
    
    # Показываем prompt
    console.print("[bold yellow]Prompt (инструкция + контекст + вопрос):[/bold yellow]")
    console.print(Panel(prompt, title="Full Prompt", border_style="cyan"))
    
    # Показываем контекст отдельно
    console.print("\n[bold yellow]Context (retrieved chunks):[/bold yellow]")
    for i, chunk in enumerate(retrieved_chunks, 1):
        console.print(f"\n  [Источник {i}] (score: {chunk.score:.4f})")
        console.print(f"  {chunk.text}")
    
    # Показываем ответ
    console.print("\n[bold yellow]Raw Response:[/bold yellow]")
    console.print(Panel(answer, title="Generated Answer", border_style="green"))
    
    return prompt, answer

prompt_demo, answer_demo = demonstrate_generation()


# 9️⃣ Evaluation & Metrics (ключевой блок)

**Исследовательский фокус**: Precision@K, влияние chunk_size, overlap, K, reranking.

Сравнение конфигураций в виде таблиц + краткие выводы.


In [ ]:
# ============================================
# Модуль: RAGAS Adapters (LangChain-совместимые)
# ============================================

# Опциональный импорт LangChain
try:
    from langchain_core.language_models.llms import LLM
    from langchain_core.embeddings import Embeddings
    from langchain_core.callbacks.manager import CallbackManagerForLLMRun
    from pydantic import PrivateAttr
    LANGCHAIN_AVAILABLE = True
except ImportError:
    LANGCHAIN_AVAILABLE = False
    # Создаём заглушки для типов
    LLM = object
    Embeddings = object
    CallbackManagerForLLMRun = Optional
    PrivateAttr = lambda x: None

# Проверяем, что LLMClient и EmbeddingService уже определены
if 'LLMClient' not in globals():
    raise NameError("Класс 'LLMClient' не определён. Пожалуйста, выполните ячейку с LLMClient.")
if 'EmbeddingService' not in globals():
    raise NameError("Класс 'EmbeddingService' не определён. Пожалуйста, выполните ячейку с EmbeddingService.")

if LANGCHAIN_AVAILABLE:
    class GigaChatLLMAdapter(LLM):
        """
        LangChain-совместимая обёртка для GigaChat LLMClient.
        Используется RAGAS для оценки метрик.
        """
        
        def __init__(
            self,
            llm_client: LLMClient,
            **kwargs
        ):
            super().__init__(**kwargs)
            self._llm_client = llm_client
        
        @property
        def _llm_type(self) -> str:
            return "gigachat"
        
        def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
        ) -> str:
            return self._llm_client.generate_answer(prompt)
        
        @property
        def _identifying_params(self) -> dict:
            return {
                "model": self._llm_client.model,
                "temperature": self._llm_client.temperature,
                "max_tokens": self._llm_client.max_tokens,
            }


if LANGCHAIN_AVAILABLE:
    class GigaChatEmbeddingsAdapter(Embeddings):
        """
        LangChain-совместимая обёртка для GigaChat EmbeddingService.
        Используется RAGAS для оценки метрик.
        """
        
        def __init__(
            self,
            embedding_service: EmbeddingService,
            **kwargs
        ):
            super().__init__(**kwargs)
            self._embedding_service = embedding_service
        
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return self._embedding_service.generate_embeddings(texts)
        
        def embed_query(self, text: str) -> List[float]:
            embeddings = self._embedding_service.generate_embeddings([text])
            return embeddings[0] if embeddings else []

# Глобальный флаг для проверки доступности адаптеров
RAGAS_ADAPTERS_AVAILABLE = LANGCHAIN_AVAILABLE

if RAGAS_ADAPTERS_AVAILABLE:
    print("✅ RAGAS LangChain адаптеры реализованы")
else:
    print("[yellow]⚠️  LangChain не установлен. RAGAS LangChain адаптеры недоступны.[/yellow]")
    print("[cyan]💡 Для использования реального RAGAS установите: pip install langchain-core langchain-community ragas datasets[/cyan]")


In [ ]:
# ============================================
# Модуль: MetricsCollector
# ============================================

class MetricsCollector:
    """
    Сборщик метрик для оценки качества системы.
    
    Зачем нужна:
    - Расчёт Precision@K для retrieved результатов
    - Сбор latency метрик (retrieval, generation, end-to-end)
    - Сбор throughput метрик (QPS)
    
    Вход: retrieved_chunks, ground_truth_relevant, k
    Выход: precision_at_k (float)
    """
    
    def __init__(self):
        pass
    
    def calculate_precision_at_k(
        self,
        retrieved_chunks: List[RetrievedChunk],
        ground_truth_relevant: List[str],
        k: int
    ) -> float:
        """
        Рассчитывает Precision@K.
        
        Precision@K = (количество релевантных чанков в топ-K) / K
        """
        if not retrieved_chunks or not ground_truth_relevant:
            return 0.0
        
        # Берём топ-K чанков
        top_k_chunks = retrieved_chunks[:k]
        
        # Подсчитываем количество релевантных чанков в топ-K
        relevant_count = sum(
            1 for chunk in top_k_chunks
            if chunk.id in ground_truth_relevant
        )
        
        # Precision@K = количество релевантных / K
        precision = relevant_count / len(top_k_chunks)
        
        return precision

# Импорты для реального RAGAS (на уровне модуля)
try:
    from ragas import evaluate
    from datasets import Dataset
    # В новых версиях RAGAS (0.4+) метрики - это классы, которые нужно импортировать из модулей
    try:
        # Новый способ (ragas >= 0.4): импортируем классы из модулей collections
        from ragas.metrics.collections.faithfulness import Faithfulness
        from ragas.metrics.collections.answer_relevancy import AnswerRelevancy
    except ImportError:
        # Старый способ (ragas < 0.4): импортируем классы напрямую
        from ragas.metrics import faithfulness as Faithfulness
        from ragas.metrics import answer_relevancy as AnswerRelevancy
    RAGAS_AVAILABLE = True
except ImportError:
    RAGAS_AVAILABLE = False

class RAGASEvaluator:
    """
    Оценщик качества ответов через RAGAS.
    
    Зачем нужна:
    - Расчёт Faithfulness (насколько ответ основан на контексте)
    - Расчёт Answer Relevancy (насколько ответ релевантен запросу)
    - Интеграция с RAGAS библиотекой
    """
    
    def __init__(
        self,
        mock_mode: bool = False,
        llm_adapter=None,
        embeddings_adapter=None
    ):
        """
        Инициализация RAGASEvaluator.
        
        Args:
            mock_mode: Если True, используется мок-режим (без реальных вызовов RAGAS)
            llm_adapter: LangChain-совместимый LLM адаптер (для реального RAGAS)
            embeddings_adapter: LangChain-совместимый Embeddings адаптер (для реального RAGAS)
        """
        self.mock_mode = mock_mode
        self.llm_adapter = llm_adapter
        self.embeddings_adapter = embeddings_adapter
        
        if not self.mock_mode:
            if not RAGAS_AVAILABLE:
                print("Warning: ragas not installed. RAGASEvaluator will operate in mock mode.")
                self.ragas_available = False
                self.mock_mode = True
            elif not llm_adapter or not embeddings_adapter:
                print("Warning: LLM or Embeddings adapter not provided. RAGASEvaluator will operate in mock mode.")
                self.ragas_available = False
                self.mock_mode = True
            else:
                self.ragas_available = True
        else:
            self.ragas_available = False
    
    def evaluate_faithfulness(
        self,
        question: str,
        answer: str,
        contexts: List[str]
    ) -> float:
        """
        Рассчитывает Faithfulness score через RAGAS.
        
        Faithfulness измеряет, насколько ответ основан на предоставленном контексте.
        Цель проекта: ≥ 0.85
        
        Args:
            question: Вопрос пользователя
            answer: Сгенерированный ответ
            contexts: Список контекстов (тексты retrieved чанков)
            
        Returns:
            Faithfulness score (0.0-1.0)
        """
        if self.mock_mode:
            # В мок-режиме возвращаем фиксированное значение для тестов
            # Проверяем, что ответ содержит текст из контекстов
            answer_lower = answer.lower()
            contexts_text = " ".join(contexts).lower()
            
            # Упрощённая проверка: если ответ содержит текст из контекстов, faithfulness высокий
            if any(context.lower() in answer_lower for context in contexts):
                return 0.90  # Высокий faithfulness
            else:
                return 0.50  # Низкий faithfulness
        
        # Реальная интеграция с RAGAS
        if not self.ragas_available:
            raise RuntimeError("RAGAS not available. Check installation and adapters.")
        
        try:
            # Создаём dataset для RAGAS
            # RAGAS ожидает: contexts - список списков, где каждый элемент - список контекстов для одного примера
            dataset_dict = {
                "question": [question],
                "answer": [answer],
                "contexts": [contexts]  # contexts уже список строк, оборачиваем в список для одного примера
            }
            dataset = Dataset.from_dict(dataset_dict)
            
            # Выполняем оценку faithfulness
            # В RAGAS метрики из ragas.metrics уже являются экземплярами классов
            # создаём новый экземпляр через класс без параметров (используются дефолты)
            # llm и embeddings передаются через evaluate()
            # Используем глобальные импорты из уровня модуля
            import ragas.metrics as ragas_metrics
            faithfulness_obj = ragas_metrics.faithfulness
            faithfulness_class = type(faithfulness_obj)
            faithfulness_metric = faithfulness_class()
            result = evaluate(
                dataset=dataset,
                metrics=[faithfulness_metric],
                llm=self.llm_adapter,
                embeddings=self.embeddings_adapter
            )
            
            # Извлекаем score (результат - DataFrame с одной строкой)
            faithfulness_score = result["faithfulness"].iloc[0] if hasattr(result, "iloc") else result["faithfulness"][0]
            return float(faithfulness_score)
        except Exception as e:
            print(f"Error evaluating faithfulness with RAGAS: {e}")
            # Fallback к mock mode при ошибке
            return 0.75
    
    def evaluate_answer_relevancy(
        self,
        question: str,
        answer: str,
        contexts: List[str]
    ) -> float:
        """
        Рассчитывает Answer Relevancy score через RAGAS.
        
        Answer Relevancy измеряет, насколько ответ релевантен вопросу.
        Цель проекта: ≥ 0.80
        
        Args:
            question: Вопрос пользователя
            answer: Сгенерированный ответ
            contexts: Список контекстов (тексты retrieved чанков)
            
        Returns:
            Answer Relevancy score (0.0-1.0)
        """
        if self.mock_mode:
            # В мок-режиме возвращаем фиксированное значение для тестов
            # Проверяем, что ответ содержит ключевые слова из вопроса
            question_lower = question.lower()
            answer_lower = answer.lower()
            
            # Упрощённая проверка: если ответ содержит ключевые слова из вопроса, relevancy высокий
            question_keywords = set(question_lower.split())
            answer_keywords = set(answer_lower.split())
            
            overlap = len(question_keywords.intersection(answer_keywords))
            if overlap > 0:
                return 0.85  # Высокий relevancy
            else:
                return 0.60  # Низкий relevancy
        
        # Реальная интеграция с RAGAS
        if not self.ragas_available:
            raise RuntimeError("RAGAS not available. Check installation and adapters.")
        
        try:
            # Создаём dataset для RAGAS
            # RAGAS ожидает: contexts - список списков, где каждый элемент - список контекстов для одного примера
            dataset_dict = {
                "question": [question],
                "answer": [answer],
                "contexts": [contexts]  # contexts уже список строк, оборачиваем в список для одного примера
            }
            dataset = Dataset.from_dict(dataset_dict)
            
            # Выполняем оценку answer_relevancy
            # В RAGAS метрики из ragas.metrics уже являются экземплярами классов
            # создаём новый экземпляр через класс без параметров (используются дефолты)
            # llm и embeddings передаются через evaluate()
            # Используем глобальные импорты из уровня модуля
            import ragas.metrics as ragas_metrics
            answer_relevancy_obj = ragas_metrics.answer_relevancy
            answer_relevancy_class = type(answer_relevancy_obj)
            answer_relevancy_metric = answer_relevancy_class()
            result = evaluate(
                dataset=dataset,
                metrics=[answer_relevancy_metric],
                llm=self.llm_adapter,
                embeddings=self.embeddings_adapter
            )
            
            # Извлекаем score (результат - DataFrame с одной строкой)
            relevancy_score = result["answer_relevancy"].iloc[0] if hasattr(result, "iloc") else result["answer_relevancy"][0]
            return float(relevancy_score)
        except Exception as e:
            print(f"Error evaluating answer_relevancy with RAGAS: {e}")
            # Fallback к mock mode при ошибке
            return 0.75
    
    def evaluate_all(
        self,
        question: str,
        answer: str,
        contexts: List[str],
        ground_truth: Optional[str] = None
    ) -> Dict[str, float]:
        """Вычисляет все метрики RAGAS."""
        faithfulness = self.evaluate_faithfulness(question, answer, contexts)
        answer_relevancy = self.evaluate_answer_relevancy(question, answer, contexts)
        
        return {
            "faithfulness": faithfulness,
            "answer_relevancy": answer_relevancy
        }

print("✅ Evaluation & Metrics реализованы")


In [ ]:
# ============================================
# ИНСТРУКЦИЯ: Как задать запрос для Production Pipeline
# ============================================

# Вариант 1: Задать запрос через переменную (рекомендуется для Jupyter/Colab)
USER_QUERY = "Какой SLA у сервиса платежей?"  # Измените на ваш запрос

# Вариант 2: Передать запрос как параметр функции
# result = run_production_pipeline("Ваш запрос здесь")

print("✅ Запрос задан. Теперь выполните следующую ячейку для запуска pipeline.")


# 🚀 Production Pipeline — Полный пайплайн с реальными данными

**Реальный запрос от пользователя → Полная индексация → Production режим → Метрики**

Этот раздел демонстрирует работу полного пайплайна в production режиме:
- Реальная индексация всех документов в Qdrant Cloud
- Production режим для EmbeddingService и LLMClient (GigaChat API)
- Обработка пользовательского запроса
- Полный вывод метрик и результатов


In [ ]:
# ============================================
# Production Pipeline — Полный пайплайн
# ============================================
import time
from datetime import datetime

def run_production_pipeline(user_query: str = None):
    """
    Полный пайплайн в production режиме:
    1. Запрос от пользователя
    2. Загрузка всех документов
    3. Индексация в Qdrant Cloud
    4. Retrieval → Reranking → Generation (production режим)
    5. Вывод метрик и результатов
    
    Args:
        user_query: Запрос пользователя. Если не указан, будет использован интерактивный ввод или значение из переменной USER_QUERY
    """
    # Проверяем, что все необходимые классы определены
    required_classes = ['DocumentLoader', 'Chunker', 'EmbeddingService', 'QdrantIndexer', 
                       'Retriever', 'Reranker', 'PromptBuilder', 'LLMClient', 
                       'MetricsCollector', 'RAGASEvaluator']
    missing_classes = [cls for cls in required_classes if cls not in globals()]
    if missing_classes:
        raise NameError(
            f"Не определены классы: {', '.join(missing_classes)}. "
            f"Пожалуйста, выполните все предыдущие ячейки с определениями модулей."
        )
    
    # Проверяем наличие LOCAL_DATA_PATH
    if 'LOCAL_DATA_PATH' not in globals():
        raise NameError(
            "Переменная LOCAL_DATA_PATH не определена. "
            "Пожалуйста, выполните ячейку 6 (Данные и домен) перед этой ячейкой."
        )
    
    console.print("\n[bold cyan]╔══════════════════════════════════════════════════════════╗[/bold cyan]")
    console.print("[bold cyan]║   🚀 PRODUCTION PIPELINE — Полный пайплайн               ║[/bold cyan]")
    console.print("[bold cyan]╚══════════════════════════════════════════════════════════╝[/bold cyan]\n")
    
    # ============================================
    # Шаг 1: Запрос от пользователя
    # ============================================
    console.print("[bold yellow]📝 Шаг 1: Ввод запроса[/bold yellow]")
    
    # Получаем запрос: из параметра функции или интерактивного ввода
    if user_query is None:
        # Сначала пробуем интерактивный ввод
        try:
            user_query = input("Введите ваш запрос: ")
            # Если пользователь ввёл пустую строку, пробуем fallback
            if not user_query or not user_query.strip():
                raise ValueError("Empty input")
        except (EOFError, KeyboardInterrupt, ValueError):
            # Если input() не работает или ввод пустой, пробуем USER_QUERY
            if 'USER_QUERY' in globals() and globals()['USER_QUERY'] and globals()['USER_QUERY'].strip():
                user_query = globals()['USER_QUERY']
                console.print(f"[cyan]Используется запрос из переменной USER_QUERY: {user_query}[/cyan]")
            else:
                # Если и USER_QUERY нет, используем дефолтный запрос
                user_query = "Какой SLA у сервиса платежей?"
                console.print(f"[yellow]⚠️  Интерактивный ввод недоступен. Используется дефолтный запрос: {user_query}[/yellow]")
                console.print("[cyan]💡 Подсказка: Задайте запрос через параметр функции или переменную USER_QUERY[/cyan]")
                console.print("[cyan]   Пример: USER_QUERY = 'Ваш запрос' или run_production_pipeline('Ваш запрос')[/cyan]")
    elif isinstance(user_query, str) and not user_query.strip():
        # Если передан пустой строковый параметр, пробуем интерактивный ввод
        try:
            user_query = input("Введите ваш запрос: ")
            if not user_query or not user_query.strip():
                raise ValueError("Empty input")
        except (EOFError, KeyboardInterrupt, ValueError):
            # Fallback на USER_QUERY или дефолтный запрос
            if 'USER_QUERY' in globals() and globals()['USER_QUERY'] and globals()['USER_QUERY'].strip():
                user_query = globals()['USER_QUERY']
                console.print(f"[cyan]Используется запрос из переменной USER_QUERY: {user_query}[/cyan]")
            else:
                user_query = "Какой SLA у сервиса платежей?"
                console.print(f"[yellow]⚠️  Интерактивный ввод недоступен. Используется дефолтный запрос: {user_query}[/yellow]")
    
    # Финальная проверка на пустой запрос
    if not user_query or (isinstance(user_query, str) and not user_query.strip()):
        console.print("[red]❌ Запрос не может быть пустым![/red]")
        console.print("[cyan]💡 Задайте запрос через параметр: run_production_pipeline('Ваш запрос')[/cyan]")
        console.print("[cyan]   Или установите переменную: USER_QUERY = 'Ваш запрос'[/cyan]")
        return None
    
    console.print(f"[green]✅ Запрос получен: {user_query}[/green]\n")
    
    # ============================================
    # Шаг 2: Загрузка документов
    # ============================================
    console.print("[bold yellow]📚 Шаг 2: Загрузка документов[/bold yellow]")
    start_time = time.time()
    
    try:
        loader = DocumentLoader()
        all_documents = []
        
        # Загружаем документы только из папок hr и it
        categories = ['hr', 'it']
        for category in categories:
            category_path = LOCAL_DATA_PATH / category
            if category_path.exists():
                docs = loader.load_documents(str(category_path))
                all_documents.extend(docs)
                console.print(f"  📁 {category}: {len(docs)} документов")
            else:
                console.print(f"  ⚠️  Папка {category} не найдена: {category_path}")
        
        if not all_documents:
            console.print("[yellow]⚠️  Документы не найдены. Проверьте путь LOCAL_DATA_PATH[/yellow]")
            return None
        
        load_time = time.time() - start_time
        console.print(f"[green]✅ Загружено документов: {len(all_documents)} (время: {load_time:.2f}с)[/green]\n")
        
    except Exception as e:
        console.print(f"[red]❌ Ошибка при загрузке документов: {e}[/red]")
        return None
    
    # ============================================
    # Шаг 3: Чанкинг документов
    # ============================================
    console.print("[bold yellow]✂️  Шаг 3: Чанкинг документов[/bold yellow]")
    start_time = time.time()
    
    try:
        chunker = Chunker()
        chunks = chunker.chunk_documents(
            all_documents,
            chunk_size=300,  # Оптимальный размер
            overlap_percent=0.25  # 25% overlap
        )
        chunk_time = time.time() - start_time
        
        console.print(f"  📊 Создано чанков: {len(chunks)}")
        console.print(f"  📏 Средний размер: {sum(c.text_length for c in chunks) / len(chunks):.1f} токенов")
        console.print(f"[green]✅ Чанкинг завершён (время: {chunk_time:.2f}с)[/green]\n")
        
    except Exception as e:
        console.print(f"[red]❌ Ошибка при чанкинге: {e}[/red]")
        return None
    
    # ============================================
    # Шаг 4: Проверка коллекции в Qdrant Cloud
    # ============================================
    console.print("[bold yellow]🔍 Шаг 4: Проверка коллекции в Qdrant Cloud[/bold yellow]")
    
    # Имя коллекции для production
    collection_name = "neuro_docs_production"
    
    # Подключаемся к Qdrant
    try:
        # Используем Qdrant Cloud (из секретов)
        if 'QDRANT_URL' not in globals() or not QDRANT_URL:
            console.print("[red]❌ QDRANT_URL не найден. Установите секрет QDRANT_URL[/red]")
            return None
        
        qdrant_url = QDRANT_URL.strip() if QDRANT_URL else None
        qdrant_api_key = QDRANT_API_KEY.strip() if 'QDRANT_API_KEY' in globals() and QDRANT_API_KEY else None
        
        if qdrant_url:
            if qdrant_api_key:
                qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
            else:
                qdrant_client = QdrantClient(url=qdrant_url)
            console.print(f"  🔗 Подключено к Qdrant Cloud: {qdrant_url[:50]}...")
        else:
            console.print("[yellow]⚠️  QDRANT_URL не найден, используется in-memory режим[/yellow]")
            qdrant_client = QdrantClient(":memory:")
        
        # Проверяем существование коллекции и количество точек
        collection_exists = False
        points_count = 0
        need_indexing = True
        
        try:
            collections = qdrant_client.get_collections()
            collection_names = [c.name for c in collections.collections]
            
            if collection_name in collection_names:
                collection_exists = True
                # Проверяем количество точек в коллекции
                try:
                    count_result = qdrant_client.count(collection_name)
                    points_count = count_result.count
                except Exception as e:
                    # Если count не работает, пробуем get_collection
                    try:
                        collection_info = qdrant_client.get_collection(collection_name)
                        points_count = collection_info.points_count
                    except:
                        points_count = 0
                
                # Если коллекция существует и содержит точки (хотя бы 80% от количества чанков),
                # считаем что индексация не нужна
                if points_count > 0:
                    chunks_count = len(chunks)
                    # Если точек больше 80% от чанков, считаем что коллекция актуальна
                    if points_count >= chunks_count * 0.8:
                        need_indexing = False
                        console.print(f"  ✅ Коллекция '{collection_name}' уже существует")
                        console.print(f"  📊 Текущий размер коллекции: {points_count} точек")
                        console.print(f"  📊 Количество чанков: {chunks_count}")
                        console.print(f"  ⏭️  Пропускаем генерацию embeddings и индексацию (используем существующую коллекцию)\n")
                    else:
                        console.print(f"  ⚠️  Коллекция существует, но содержит меньше точек ({points_count} < {chunks_count * 0.8})")
                        console.print(f"  🔄 Будет выполнена переиндексация\n")
                else:
                    console.print(f"  ℹ️  Коллекция '{collection_name}' существует, но пуста")
                    console.print(f"  🔄 Будет выполнена индексация\n")
            else:
                console.print(f"  ℹ️  Коллекция '{collection_name}' не существует")
                console.print(f"  🔄 Будет создана новая коллекция и выполнена индексация\n")
        except Exception as e:
            console.print(f"  ⚠️  Ошибка при проверке коллекции: {e}")
            console.print(f"  🔄 Будет выполнена индексация\n")
        
    except Exception as e:
        console.print(f"[red]❌ Ошибка при подключении к Qdrant: {e}[/red]")
        return None
    
    # ============================================
    # Шаг 5: Генерация embeddings (если нужно)
    # ============================================
    embeddings = None
    actual_embedding_dim = 1536  # Дефолтная размерность
    embedding_service = None
    embedding_time = 0.0  # Время генерации embeddings (если пропущено, будет 0)
    index_time = 0.0  # Время индексации (если пропущено, будет 0)
    
    if need_indexing:
        console.print("[bold yellow]🧠 Шаг 5: Генерация embeddings (Production режим)[/bold yellow]")
        start_time = time.time()
        
        try:
            # Production режим: используем реальный GigaChat API
            embedding_service = EmbeddingService(
                embedding_dim=1536,
                batch_size=10,
                mock_mode=False  # Production режим!
            )
            
            # Генерируем embeddings для всех чанков
            chunk_texts = [chunk.text for chunk in chunks]
            embeddings = embedding_service.generate_embeddings(chunk_texts)
            
            embedding_time = time.time() - start_time
            
            # Определяем размерность embeddings динамически
            actual_embedding_dim = len(embeddings[0]) if embeddings else 1536
            
            console.print(f"  📊 Сгенерировано embeddings: {len(embeddings)}")
            console.print(f"  📏 Размерность: {actual_embedding_dim}")
            console.print(f"[green]✅ Embeddings сгенерированы (время: {embedding_time:.2f}с, скорость: {len(embeddings)/embedding_time:.1f} чанков/с)[/green]\n")
            
        except Exception as e:
            console.print(f"[red]❌ Ошибка при генерации embeddings: {e}[/red]")
            console.print("[yellow]💡 Попробуйте проверить GIGACHAT_AUTH_KEY и GIGACHAT_SCOPE в секретах[/yellow]")
            return None
    else:
        # Если индексация не нужна, всё равно создаём embedding_service для retrieval
        embedding_service = EmbeddingService(
            embedding_dim=1536,
            batch_size=10,
            mock_mode=False
        )
        console.print("[bold yellow]🧠 Шаг 5: Генерация embeddings[/bold yellow]")
        console.print("  ⏭️  Пропущено (используются существующие embeddings в Qdrant)\n")
        embedding_time = 0.0  # Время = 0, так как шаг пропущен
    
    # ============================================
    # Шаг 6: Индексация в Qdrant Cloud (если нужно)
    # ============================================
    if need_indexing:
        console.print("[bold yellow]☁️  Шаг 6: Индексация в Qdrant Cloud[/bold yellow]")
        start_time = time.time()
        
        try:
            # Проверяем коллекцию: если существует с неправильной размерностью, удаляем
            if collection_exists:
                try:
                    existing_dim = None
                    try:
                        collection_info = qdrant_client.get_collection(collection_name)
                        existing_dim = collection_info.config.params.vectors.size
                    except Exception:
                        pass
                    
                    if existing_dim is not None and existing_dim != actual_embedding_dim:
                        console.print(f"  ⚠️  Коллекция существует с размерностью {existing_dim}, требуется {actual_embedding_dim}")
                        console.print(f"  🔄 Удаляем коллекцию и создаём новую с правильной размерностью...")
                        qdrant_client.delete_collection(collection_name)
                        console.print(f"  ✅ Коллекция удалена")
                    elif existing_dim is None:
                        # Не удалось проверить размерность - удаляем коллекцию для пересоздания
                        console.print(f"  ⚠️  Не удалось проверить размерность коллекции, пересоздаём...")
                        qdrant_client.delete_collection(collection_name)
                        console.print(f"  ✅ Старая коллекция удалена")
                except Exception as e:
                    console.print(f"  ⚠️  Ошибка при проверке размерности: {e}")
                    try:
                        qdrant_client.delete_collection(collection_name)
                        console.print(f"  ✅ Коллекция удалена перед пересозданием")
                    except:
                        pass  # Коллекция может не существовать
            
            # Индексируем чанки с правильной размерностью (определяется из embeddings)
            indexer = QdrantIndexer(
                qdrant_client=qdrant_client,
                collection_name=collection_name,
                embedding_dim=actual_embedding_dim  # Используем размерность из embeddings
            )
            
            indexed_count = indexer.index_chunks(chunks, embeddings)
            index_time = time.time() - start_time
            
            # Проверяем коллекцию после индексации
            try:
                count_result = qdrant_client.count(collection_name)
                points_count = count_result.count
            except:
                points_count = indexed_count
            
            console.print(f"  📊 Проиндексировано чанков: {indexed_count}")
            console.print(f"  📈 Размер коллекции: {points_count} точек")
            console.print(f"[green]✅ Индексация завершена (время: {index_time:.2f}с, скорость: {indexed_count/index_time:.1f} чанков/с)[/green]\n")
            
        except Exception as e:
            console.print(f"[red]❌ Ошибка при индексации: {e}[/red]")
            return None
    else:
        console.print("[bold yellow]☁️  Шаг 6: Индексация в Qdrant Cloud[/bold yellow]")
        console.print("  ⏭️  Пропущено (коллекция уже существует и актуальна)\n")
        index_time = 0.0  # Время = 0, так как шаг пропущен
    
    # ============================================
    # Шаг 7: Retrieval
    # ============================================
    console.print("[bold yellow]🔍 Шаг 7: Semantic Search (Retrieval)[/bold yellow]")
    start_time = time.time()
    
    try:
        retriever = Retriever(
            qdrant_client=qdrant_client,
            embedding_service=embedding_service,
            collection_name=collection_name
        )
        
        retrieved_chunks = retriever.retrieve(user_query, k=5)
        retrieval_time = time.time() - start_time
        
        console.print(f"  📊 Найдено чанков: {len(retrieved_chunks)}")
        console.print(f"  ⭐ Средний score: {sum(c.score for c in retrieved_chunks) / len(retrieved_chunks):.4f}" if retrieved_chunks else "  ⭐ Средний score: N/A")
        console.print(f"[green]✅ Retrieval завершён (время: {retrieval_time:.3f}с)[/green]\n")
        
    except Exception as e:
        console.print(f"[red]❌ Ошибка при retrieval: {e}[/red]")
        return None
    
    # ============================================
    # Шаг 8: Reranking
    # ============================================
    console.print("[bold yellow]🎯 Шаг 8: Reranking[/bold yellow]")
    start_time = time.time()
    
    try:
        reranker = Reranker()
        reranked_chunks = reranker.rerank(user_query, retrieved_chunks, top_k=3)
        rerank_time = time.time() - start_time
        
        console.print(f"  📊 Отобрано чанков после reranking: {len(reranked_chunks)}")
        console.print(f"  ⭐ Средний rerank score: {sum(c.rerank_score for c in reranked_chunks) / len(reranked_chunks):.4f}" if reranked_chunks else "  ⭐ Средний rerank score: N/A")
        console.print(f"[green]✅ Reranking завершён (время: {rerank_time:.3f}с)[/green]\n")
        
    except Exception as e:
        console.print(f"[red]❌ Ошибка при reranking: {e}[/red]")
        return None
    
    # ============================================
    # Шаг 9: Generation (Production режим)
    # ============================================
    console.print("[bold yellow]💬 Шаг 9: Generation ответа (Production режим)[/bold yellow]")
    start_time = time.time()
    
    try:
        prompt_builder = PromptBuilder()
        prompt = prompt_builder.build_prompt(user_query, reranked_chunks)
        
        # Production режим: используем реальный GigaChat API
        # LLMClient автоматически переключится на mock режим при ошибках API
        llm_client = LLMClient(mock_mode=False)  # Production режим!
        answer = llm_client.generate_answer(prompt)
        
        generation_time = time.time() - start_time
        console.print(f"[green]✅ Ответ сгенерирован (время: {generation_time:.2f}с)[/green]\n")
        
    except Exception as e:
        console.print(f"[red]❌ Ошибка при генерации ответа: {e}[/red]")
        console.print("[yellow]💡 Попробуйте проверить GIGACHAT_AUTH_KEY и GIGACHAT_SCOPE в секретах[/yellow]")
        return None
    
    # ============================================
    # Шаг 10: Метрики
    # ============================================
    console.print("[bold yellow]📊 Шаг 10: Вычисление метрик[/bold yellow]")
    
    try:
        metrics_collector = MetricsCollector()
        # Инициализируем RAGASEvaluator для реального режима, если возможно
        # Для этого нужны LLMClient и EmbeddingService в реальном режиме
        llm_for_ragas = LLMClient(mock_mode=False)
        embedding_for_ragas = EmbeddingService(mock_mode=False)
        
        # Проверяем, доступны ли адаптеры LangChain
        try:
            from langchain_core.language_models.llms import LLM
            from langchain_core.embeddings import Embeddings
            from ragas import evaluate
            from datasets import Dataset
            # Метрики уже импортированы на уровне модуля (Faithfulness, AnswerRelevancy)
            
            # Импортируем адаптеры (если они определены в ноутбуке)
            if 'GigaChatLLMAdapter' in globals() and 'GigaChatEmbeddingsAdapter' in globals():
                llm_adapter = GigaChatLLMAdapter(llm_client=llm_for_ragas)
                embeddings_adapter = GigaChatEmbeddingsAdapter(embedding_service=embedding_for_ragas)
                ragas_evaluator = RAGASEvaluator(
                    mock_mode=False,
                    llm_adapter=llm_adapter,
                    embeddings_adapter=embeddings_adapter
                )
                console.print("[green]✅ RAGASEvaluator инициализирован в реальном режиме.[/green]")
            else:
                ragas_evaluator = RAGASEvaluator(mock_mode=True)
                console.print("[yellow]⚠️  LangChain адаптеры не найдены. RAGASEvaluator будет работать в mock режиме.[/yellow]")
        except ImportError as e:
            ragas_evaluator = RAGASEvaluator(mock_mode=True)
            console.print(f"[yellow]⚠️  Не удалось импортировать RAGAS или LangChain: {e}. RAGASEvaluator будет работать в mock режиме.[/yellow]")
        except Exception as e:
            ragas_evaluator = RAGASEvaluator(mock_mode=True)
            console.print(f"[yellow]⚠️  Ошибка при инициализации реального RAGAS: {e}. Используется mock режим.[/yellow]")
        
        contexts = [chunk.text for chunk in reranked_chunks]
        ragas_metrics = ragas_evaluator.evaluate_all(user_query, answer, contexts)
        
        console.print("[green]✅ Метрики вычислены[/green]\n")
        
    except Exception as e:
        console.print(f"[yellow]⚠️  Ошибка при вычислении метрик: {e}[/yellow]")
        ragas_metrics = {"faithfulness": 0.0, "answer_relevancy": 0.0}
    
    # ============================================
    # Шаг 11: Вывод результатов
    # ============================================
    console.print("[bold cyan]╔══════════════════════════════════════════════════════════╗[/bold cyan]")
    console.print("[bold cyan]║                    📋 РЕЗУЛЬТАТЫ                         ║[/bold cyan]")
    console.print("[bold cyan]╚══════════════════════════════════════════════════════════╝[/bold cyan]\n")
    
    # Запрос
    console.print(Panel(f"[bold]{user_query}[/bold]", title="👤 Ваш запрос", border_style="cyan"))
    
    # Ответ
    console.print(Panel(answer, title="🤖 Ответ системы", border_style="green"))
    
    # Источники
    console.print("\n[bold yellow]📚 Источники (Retrieved & Reranked):[/bold yellow]")
    sources_table = Table(show_header=True, header_style="bold magenta")
    sources_table.add_column("№", style="dim", width=3)
    sources_table.add_column("Score", justify="right", style="cyan", width=8)
    sources_table.add_column("Rerank Score", justify="right", style="green", width=12)
    sources_table.add_column("Источник", style="yellow", width=20)
    sources_table.add_column("Текст (первые 60 символов)", style="white", width=60)
    
    for i, chunk in enumerate(reranked_chunks, 1):
        source_name = chunk.metadata.get("file_name", chunk.metadata.get("category", "unknown"))
        sources_table.add_row(
            str(i),
            f"{chunk.score:.4f}",
            f"{chunk.rerank_score:.4f}",
            source_name,
            chunk.text[:60] + "..." if len(chunk.text) > 60 else chunk.text
        )
    
    console.print(sources_table)
    
    # Метрики
    console.print("\n[bold yellow]📊 Метрики качества:[/bold yellow]")
    metrics_table = Table(show_header=True, header_style="bold magenta")
    metrics_table.add_column("Метрика", style="cyan", width=30)
    metrics_table.add_column("Значение", justify="right", style="green", width=15)
    metrics_table.add_column("Описание", style="white")
    
    metrics_table.add_row("Faithfulness", f"{ragas_metrics['faithfulness']:.3f}",
                          "Насколько ответ основан на контексте (0-1)")
    metrics_table.add_row("Answer Relevancy", f"{ragas_metrics['answer_relevancy']:.3f}",
                          "Релевантность ответа запросу (0-1)")
    
    console.print(metrics_table)
    
    # Производительность
    total_time = load_time + chunk_time + embedding_time + index_time + retrieval_time + rerank_time + generation_time
    console.print("\n[bold yellow]⏱️  Производительность:[/bold yellow]")
    perf_table = Table(show_header=True, header_style="bold magenta")
    perf_table.add_column("Этап", style="cyan", width=25)
    perf_table.add_column("Время", justify="right", style="yellow", width=12)
    perf_table.add_column("% от общего", justify="right", style="dim", width=12)
    
    stages = [
        ("Загрузка документов", load_time),
        ("Чанкинг", chunk_time),
        ("Генерация embeddings", embedding_time),
        ("Индексация", index_time),
        ("Retrieval", retrieval_time),
        ("Reranking", rerank_time),
        ("Generation", generation_time)
    ]
    
    for stage_name, stage_time in stages:
        percentage = (stage_time / total_time * 100) if total_time > 0 else 0
        perf_table.add_row(stage_name, f"{stage_time:.2f}с", f"{percentage:.1f}%")
    
    perf_table.add_row("[bold]Итого[/bold]", f"[bold]{total_time:.2f}с[/bold]", "100.0%")
    console.print(perf_table)
    
    # Статистика
    console.print("\n[bold yellow]📈 Статистика:[/bold yellow]")
    console.print(f"  • Всего документов: {len(all_documents)}")
    console.print(f"  • Всего чанков: {len(chunks)}")
    console.print(f"  • Retrieved чанков: {len(retrieved_chunks)}")
    console.print(f"  • Reranked чанков: {len(reranked_chunks)}")
    console.print(f"  • Размер коллекции Qdrant: {points_count} точек")
    
    console.print("\n[bold green]✅ Production Pipeline завершён успешно![/bold green]\n")
    
    return {
        "query": user_query,
        "answer": answer,
        "sources": reranked_chunks,
        "metrics": ragas_metrics,
        "performance": {
            "total_time": total_time,
            "stages": stages
        },
        "statistics": {
            "documents": len(all_documents),
            "chunks": len(chunks),
            "retrieved": len(retrieved_chunks),
            "reranked": len(reranked_chunks),
            "qdrant_points": points_count
        }
    }

# Запускаем production pipeline (используется USER_QUERY из предыдущей ячейки)
result = run_production_pipeline()


# 📊 Сравнение: без reranking vs с reranking (Production данные)

Сравнение результатов retrieval с reranking и без reranking на основе реальных данных из Production Pipeline.


In [ ]:
# ============================================
# Сравнение: без reranking vs с reranking (Production данные)
# ============================================

def compare_reranking_production(query: str = None, qdrant_client=None, collection_name: str = "neuro_docs_production"):
    """
    Сравнение результатов с reranking и без на основе реальных данных из Production Pipeline.
    
    Args:
        query: Запрос. Если не указан, используется из result production pipeline
        qdrant_client: Qdrant клиент. Если не указан, используется из production pipeline
        collection_name: Имя коллекции (по умолчанию production коллекция)
    """
    # Используем данные из production pipeline, если доступны
    if 'result' not in globals() or result is None:
        raise ValueError("Сначала выполните Production Pipeline! Переменная 'result' не найдена.")
    
    # Получаем запрос из result или параметра
    if query is None:
        query = result.get('query', 'Какой SLA у сервиса платежей?')
    
    # Получаем Qdrant клиент из production pipeline
    if qdrant_client is None:
        # Нужно получить из production pipeline или создать новый
        if 'QDRANT_URL' in globals() and QDRANT_URL:
            qdrant_url = QDRANT_URL.strip() if QDRANT_URL else None
            qdrant_api_key = QDRANT_API_KEY.strip() if 'QDRANT_API_KEY' in globals() and QDRANT_API_KEY else None
            if qdrant_url:
                if qdrant_api_key:
                    qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
                else:
                    qdrant_client = QdrantClient(url=qdrant_url)
            else:
                raise ValueError("QDRANT_URL не найден. Выполните Production Pipeline сначала.")
        else:
            raise ValueError("QDRANT_URL не найден. Выполните Production Pipeline сначала.")
    
    # Production режим для embedding service
    embedding_service = EmbeddingService(mock_mode=False)
    
    # Без reranking
    retriever = Retriever(
        qdrant_client=qdrant_client,
        embedding_service=embedding_service,
        collection_name=collection_name
    )
    chunks_without_rerank = retriever.retrieve(query, k=5)
    
    # С reranking
    reranker = Reranker()
    reranked_chunks = reranker.rerank(query, chunks_without_rerank, top_k=3)
    
    # Выводим сравнение
    console.print(f"\n[bold cyan]╔══════════════════════════════════════════════════════════╗[/bold cyan]")
    console.print(f"[bold cyan]║   📊 СРАВНЕНИЕ RERANKING (Production данные)            ║[/bold cyan]")
    console.print(f"[bold cyan]╚══════════════════════════════════════════════════════════╝[/bold cyan]")
    console.print(f"[bold]Запрос:[/bold] {query}\n")
    
    # Таблица: без reranking
    table1 = Table(title="БЕЗ Reranking", show_header=True, header_style="bold red")
    table1.add_column("№", style="dim", width=3)
    table1.add_column("Score", justify="right", style="cyan", width=10)
    table1.add_column("Источник", style="yellow", width=20)
    table1.add_column("Text Preview", style="white", width=50)
    
    for i, chunk in enumerate(chunks_without_rerank[:5], 1):
        source_name = chunk.metadata.get("file_name", chunk.metadata.get("category", "unknown"))
        table1.add_row(
            str(i),
            f"{chunk.score:.4f}",
            source_name[:20] + "..." if len(source_name) > 20 else source_name,
            chunk.text[:60] + "..." if len(chunk.text) > 60 else chunk.text
        )
    
    console.print(table1)
    
    # Таблица: с reranking
    table2 = Table(title="С Reranking", show_header=True, header_style="bold green")
    table2.add_column("№", style="dim", width=3)
    table2.add_column("Original Score", justify="right", style="cyan", width=12)
    table2.add_column("Rerank Score", justify="right", style="green", width=12)
    table2.add_column("Источник", style="yellow", width=20)
    table2.add_column("Text Preview", style="white", width=50)
    
    for i, chunk in enumerate(reranked_chunks, 1):
        source_name = chunk.metadata.get("file_name", chunk.metadata.get("category", "unknown"))
        table2.add_row(
            str(i),
            f"{chunk.score:.4f}",
            f"{chunk.rerank_score:.4f}",
            source_name[:20] + "..." if len(source_name) > 20 else source_name,
            chunk.text[:60] + "..." if len(chunk.text) > 60 else chunk.text
        )
    
    console.print(table2)
    
    # Анализ изменений
    console.print("\n[bold yellow]Анализ:[/bold yellow]")
    order_changed = False
    rerank_ids = [chunk.id for chunk in reranked_chunks]
    original_ids = [chunk.id for chunk in chunks_without_rerank[:len(reranked_chunks)]]
    
    for i, (orig_id, rerank_id) in enumerate(zip(original_ids, rerank_ids), 1):
        if orig_id != rerank_id:
            order_changed = True
            console.print(f"  Позиция {i}: изменён порядок ({orig_id[:20]}... → {rerank_id[:20]}...)")
    
    if not order_changed:
        console.print("  Порядок не изменился (для этого запроса reranking не повлиял на порядок)")
    
    # Статистика по scores
    avg_original_score = sum(c.score for c in chunks_without_rerank[:len(reranked_chunks)]) / len(reranked_chunks)
    avg_rerank_score = sum(c.rerank_score for c in reranked_chunks) / len(reranked_chunks)
    console.print(f"\n[bold]Статистика:[/bold]")
    console.print(f"  Средний original score (топ-{len(reranked_chunks)}): {avg_original_score:.4f}")
    console.print(f"  Средний rerank score: {avg_rerank_score:.4f}")
    console.print(f"  Разница: {avg_rerank_score - avg_original_score:+.4f}")
    
    return chunks_without_rerank, reranked_chunks

# Выполняем сравнение
if 'result' in globals() and result:
    chunks_no_rerank_prod, chunks_reranked_prod = compare_reranking_production()
else:
    console.print("[yellow]⚠️  Сначала выполните Production Pipeline![/yellow]")


# 🔬 Эксперименты: сравнение конфигураций (Production данные)

Сравнение разных конфигураций (K, reranking) на основе реальных данных из Production Pipeline.


In [ ]:
# ============================================
# Эксперименты: сравнение конфигураций (Production данные)
# ============================================

def run_experiments_production(query: str = None, qdrant_client=None, collection_name: str = "neuro_docs_production"):
    """
    Запускает эксперименты с разными конфигурациями на основе реальных данных.
    
    Сравнивает:
    - K: 3, 5, 8
    - reranking: on/off
    
    Args:
        query: Запрос. Если не указан, используется из result production pipeline
        qdrant_client: Qdrant клиент. Если не указан, создаётся новый
        collection_name: Имя коллекции (по умолчанию production коллекция)
    """
    # Используем данные из production pipeline
    if 'result' not in globals() or result is None:
        raise ValueError("Сначала выполните Production Pipeline! Переменная 'result' не найдена.")
    
    # Получаем запрос из result или параметра
    if query is None:
        query = result.get('query', 'Какой SLA у сервиса платежей?')
    
    # Получаем Qdrant клиент
    if qdrant_client is None:
        if 'QDRANT_URL' in globals() and QDRANT_URL:
            qdrant_url = QDRANT_URL.strip() if QDRANT_URL else None
            qdrant_api_key = QDRANT_API_KEY.strip() if 'QDRANT_API_KEY' in globals() and QDRANT_API_KEY else None
            if qdrant_url:
                if qdrant_api_key:
                    qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
                else:
                    qdrant_client = QdrantClient(url=qdrant_url)
            else:
                raise ValueError("QDRANT_URL не найден. Выполните Production Pipeline сначала.")
        else:
            raise ValueError("QDRANT_URL не найден. Выполните Production Pipeline сначала.")
    
    experiments = []
    
    # Упрощённые конфигурации (только K и reranking, без переиндексации)
    configs = [
        {"k": 3, "reranking": False},
        {"k": 5, "reranking": False},
        {"k": 3, "reranking": True},
        {"k": 5, "reranking": True},
    ]
    
    # Production режим
    embedding_service = EmbeddingService(mock_mode=False)
    llm_client = LLMClient(mock_mode=False)
    
    # Инициализируем RAGAS evaluator для реального режима
    try:
        from langchain_core.language_models.llms import LLM
        from langchain_core.embeddings import Embeddings
        from ragas import evaluate
        from datasets import Dataset
        
        if 'GigaChatLLMAdapter' in globals() and 'GigaChatEmbeddingsAdapter' in globals():
            llm_adapter = GigaChatLLMAdapter(llm_client=llm_client)
            embeddings_adapter = GigaChatEmbeddingsAdapter(embedding_service=embedding_service)
            ragas_evaluator = RAGASEvaluator(
                mock_mode=False,
                llm_adapter=llm_adapter,
                embeddings_adapter=embeddings_adapter
            )
        else:
            ragas_evaluator = RAGASEvaluator(mock_mode=True)
    except:
        ragas_evaluator = RAGASEvaluator(mock_mode=True)
    
    metrics_collector = MetricsCollector()
    
    console.print(f"\n[bold cyan]🔬 Запуск экспериментов (Production режим)[/bold cyan]")
    console.print(f"[bold]Запрос:[/bold] {query}\n")
    
    for i, config in enumerate(configs, 1):
        console.print(f"[cyan]Эксперимент {i}/{len(configs)}: K={config['k']}, reranking={'on' if config['reranking'] else 'off'}[/cyan]")
        
        try:
            # Retrieval
            retriever = Retriever(
                qdrant_client=qdrant_client,
                embedding_service=embedding_service,
                collection_name=collection_name
            )
            retrieved_chunks = retriever.retrieve(query, k=config["k"])
            
            # Reranking (если включён)
            if config["reranking"]:
                reranker = Reranker()
                reranked_chunks = reranker.rerank(query, retrieved_chunks, top_k=config["k"])
                # Используем reranked chunks
                final_chunks = reranked_chunks
            else:
                final_chunks = retrieved_chunks
            
            # Generation
            prompt_builder = PromptBuilder()
            prompt = prompt_builder.build_prompt(query, final_chunks[:3])  # Берём топ-3 для generation
            answer = llm_client.generate_answer(prompt)
            
            # Metrics (RAGAS)
            contexts = [chunk.text for chunk in final_chunks[:3]]
            ragas_metrics = ragas_evaluator.evaluate_all(query, answer, contexts)
            
            experiments.append({
                "config": config,
                "faithfulness": ragas_metrics["faithfulness"],
                "answer_relevancy": ragas_metrics["answer_relevancy"],
                "retrieved_count": len(final_chunks),
                "avg_score": sum(c.score for c in final_chunks) / len(final_chunks) if final_chunks else 0.0
            })
            
            console.print(f"  ✅ Завершён: faithfulness={ragas_metrics['faithfulness']:.3f}, relevancy={ragas_metrics['answer_relevancy']:.3f}")
            
        except Exception as e:
            console.print(f"  ❌ Ошибка: {e}")
            experiments.append({
                "config": config,
                "faithfulness": 0.0,
                "answer_relevancy": 0.0,
                "retrieved_count": 0,
                "avg_score": 0.0
            })
    
    return experiments

# Запускаем эксперименты
if 'result' in globals() and result:
    experiments_prod = run_experiments_production()
    
    # Выводим результаты
    console.print(f"\n[bold cyan]╔══════════════════════════════════════════════════════════╗[/bold cyan]")
    console.print(f"[bold cyan]║   📊 РЕЗУЛЬТАТЫ ЭКСПЕРИМЕНТОВ (Production)                ║[/bold cyan]")
    console.print(f"[bold cyan]╚══════════════════════════════════════════════════════════╝[/bold cyan]\n")
    
    table = Table(title="Сравнение конфигураций", show_header=True, header_style="bold magenta")
    table.add_column("Config", style="cyan", width=25)
    table.add_column("Faithfulness", justify="right", style="yellow", width=12)
    table.add_column("Answer Relevancy", justify="right", style="blue", width=15)
    table.add_column("Retrieved", justify="right", style="dim", width=10)
    table.add_column("Avg Score", justify="right", style="green", width=10)
    
    for exp in experiments_prod:
        config = exp["config"]
        config_str = f"K={config['k']}, rerank={'on' if config['reranking'] else 'off'}"
        table.add_row(
            config_str,
            f"{exp['faithfulness']:.3f}",
            f"{exp['answer_relevancy']:.3f}",
            str(exp['retrieved_count']),
            f"{exp['avg_score']:.4f}"
        )
    
    console.print(table)
    
    # Анализ результатов
    console.print("\n[bold yellow]📈 Анализ результатов:[/bold yellow]")
    
    # Находим лучшую конфигурацию по faithfulness
    best_faith = max(experiments_prod, key=lambda x: x['faithfulness'])
    console.print(f"\n  🏆 Лучшая конфигурация по Faithfulness:")
    console.print(f"     Config: K={best_faith['config']['k']}, rerank={'on' if best_faith['config']['reranking'] else 'off'}")
    console.print(f"     Faithfulness: {best_faith['faithfulness']:.3f}")
    
    # Находим лучшую конфигурацию по answer_relevancy
    best_relevancy = max(experiments_prod, key=lambda x: x['answer_relevancy'])
    console.print(f"\n  🏆 Лучшая конфигурация по Answer Relevancy:")
    console.print(f"     Config: K={best_relevancy['config']['k']}, rerank={'on' if best_relevancy['config']['reranking'] else 'off'}")
    console.print(f"     Answer Relevancy: {best_relevancy['answer_relevancy']:.3f}")
    
    # Сравнение с reranking и без
    with_rerank = [e for e in experiments_prod if e['config']['reranking']]
    without_rerank = [e for e in experiments_prod if not e['config']['reranking']]
    if with_rerank and without_rerank:
        avg_faith_with = sum(e['faithfulness'] for e in with_rerank) / len(with_rerank)
        avg_faith_without = sum(e['faithfulness'] for e in without_rerank) / len(without_rerank)
        console.print(f"\n  📊 Влияние reranking на Faithfulness:")
        console.print(f"     С reranking: {avg_faith_with:.3f}")
        console.print(f"     Без reranking: {avg_faith_without:.3f}")
        console.print(f"     Разница: {avg_faith_with - avg_faith_without:+.3f}")
else:
    console.print("[yellow]⚠️  Сначала выполните Production Pipeline![/yellow]")


# 🔟 Итоговый вывод (Production данные)

Что показал эксперимент с реальными данными, какие гипотезы подтверждены, что можно улучшать дальше.


In [ ]:
# ============================================
# Итоговый вывод (Production данные)
# ============================================

console.print("\n[bold cyan]╔══════════════════════════════════════════════════════════╗[/bold cyan]")
console.print("[bold cyan]║   🔟 ИТОГОВЫЙ ВЫВОД (Production Pipeline)              ║[/bold cyan]")
console.print("[bold cyan]╚══════════════════════════════════════════════════════════╝[/bold cyan]\n")

# Используем данные из production pipeline и экспериментов
if 'result' in globals() and result:
    production_metrics = result.get('metrics', {})
    production_stats = result.get('statistics', {})
    production_perf = result.get('performance', {})
    
    console.print("[bold yellow]1. Что показал Production Pipeline:[/bold yellow]")
    console.print(f"  ✅ Загружено документов: {production_stats.get('documents', 'N/A')}")
    console.print(f"  ✅ Создано чанков: {production_stats.get('chunks', 'N/A')}")
    console.print(f"  ✅ Индексировано в Qdrant: {production_stats.get('qdrant_points', 'N/A')} точек")
    console.print(f"  ✅ Retrieved чанков: {production_stats.get('retrieved', 'N/A')}")
    console.print(f"  ✅ Reranked чанков: {production_stats.get('reranked', 'N/A')}")
    
    if production_metrics:
        console.print(f"\n  📊 Метрики качества:")
        console.print(f"     • Faithfulness: {production_metrics.get('faithfulness', 0):.3f} {'✅ (≥0.85)' if production_metrics.get('faithfulness', 0) >= 0.85 else '⚠️  (<0.85)'}")
        console.print(f"     • Answer Relevancy: {production_metrics.get('answer_relevancy', 0):.3f} {'✅ (≥0.80)' if production_metrics.get('answer_relevancy', 0) >= 0.80 else '⚠️  (<0.80)'}")
    
    if production_perf.get('total_time'):
        console.print(f"\n  ⏱️  Производительность:")
        console.print(f"     • Общее время: {production_perf['total_time']:.2f}с")
        for stage_name, stage_time in production_perf.get('stages', []):
            if stage_time > 0:
                console.print(f"     • {stage_name}: {stage_time:.2f}с")
else:
    console.print("[yellow]⚠️  Данные Production Pipeline не найдены[/yellow]")

console.print("\n[bold yellow]2. Где система даёт выигрыш:[/bold yellow]")
console.print("  ✅ Production-готовность: реальные API (GigaChat, Qdrant Cloud)")
console.print("  ✅ Модульная архитектура: легко экспериментировать с конфигурациями")
console.print("  ✅ Полная метрика: RAGAS оценивает качество ответов объективно")
console.print("  ✅ Масштабируемость: Qdrant Cloud поддерживает большие объёмы данных")
console.print("  ✅ Прозрачность: видно все этапы pipeline и метрики на каждом шаге")

if 'experiments_prod' in globals() and experiments_prod:
    console.print("\n[bold yellow]3. Результаты экспериментов:[/bold yellow]")
    best_faith = max(experiments_prod, key=lambda x: x['faithfulness'])
    best_relevancy = max(experiments_prod, key=lambda x: x['answer_relevancy'])
    
    console.print(f"  🏆 Лучшая конфигурация:")
    console.print(f"     • По Faithfulness: K={best_faith['config']['k']}, rerank={'on' if best_faith['config']['reranking'] else 'off'}")
    console.print(f"     • По Answer Relevancy: K={best_relevancy['config']['k']}, rerank={'on' if best_relevancy['config']['reranking'] else 'off'}")
    
    # Сравнение reranking
    with_rerank = [e for e in experiments_prod if e['config']['reranking']]
    without_rerank = [e for e in experiments_prod if not e['config']['reranking']]
    if with_rerank and without_rerank:
        avg_faith_with = sum(e['faithfulness'] for e in with_rerank) / len(with_rerank)
        avg_faith_without = sum(e['faithfulness'] for e in without_rerank) / len(without_rerank)
        console.print(f"\n  📊 Влияние reranking:")
        console.print(f"     • С reranking: Faithfulness = {avg_faith_with:.3f}")
        console.print(f"     • Без reranking: Faithfulness = {avg_faith_without:.3f}")
        if avg_faith_with > avg_faith_without:
            console.print(f"     ✅ Reranking улучшает качество (+{avg_faith_with - avg_faith_without:.3f})")
        else:
            console.print(f"     ⚠️  Reranking не улучшает качество в данном случае")

console.print("\n[bold yellow]4. Какие гипотезы подтверждены:[/bold yellow]")
if 'result' in globals() and result:
    metrics = result.get('metrics', {})
    if metrics.get('faithfulness', 0) >= 0.85:
        console.print("  ✅ Faithfulness ≥ 0.85 достижимо со строгой инструкцией в prompt")
    if metrics.get('answer_relevancy', 0) >= 0.80:
        console.print("  ✅ Answer Relevancy ≥ 0.80 достижимо с качественным retrieval")
console.print("  ✅ Production режим работает стабильно с реальными API")
console.print("  ✅ RAGAS метрики дают объективную оценку качества")
console.print("  ✅ Модульная архитектура позволяет экспериментировать")

console.print("\n[bold yellow]5. Что можно улучшать дальше:[/bold yellow]")
console.print("  🔄 Оптимизация chunk_size и overlap для конкретного домена")
console.print("  🔄 Эксперименты с разными embedding моделями")
console.print("  🔄 Улучшение reranking (использование cross-encoder моделей)")
console.print("  🔄 Добавление метаданных фильтрации (по категории, дате)")
console.print("  🔄 Расширение метрик (Recall@K, MRR, NDCG)")
console.print("  🔄 A/B тестирование разных конфигураций в production")
console.print("  🔄 Мониторинг метрик в реальном времени")
console.print("  🔄 Кэширование частых запросов для улучшения производительности")

console.print("\n[bold green]✅ Production Pipeline демонстрация завершена![/bold green]")
console.print("\nЭтот notebook показывает:")
console.print("  • Полный пайплайн в production режиме")
console.print("  • Реальную интеграцию с GigaChat API и Qdrant Cloud")
console.print("  • Объективную оценку качества через RAGAS")
console.print("  • Сравнение конфигураций на реальных данных")
console.print("  • Готовность к использованию в production")


# 💬 Telegram Bot интеграция (опционально)

Интеграция Production Pipeline в Telegram-бот для удобного взаимодействия через мессенджер.

**Возможности:**
- Ввод запросов через Telegram
- Получение ответов с метриками качества
- Простой и надежный интерфейс

**Примечание:** Если возникают ошибки импорта aiogram, выполните:
```python
!pip install --upgrade --force-reinstall aiogram pydantic
```


In [ ]:
# ============================================
# Telegram Bot интеграция (опционально)
# ============================================

# Установка зависимостей для Telegram-бота
# Исправляем проблемы совместимости: устанавливаем совместимые версии
!pip install -q "pydantic>=2.0.0,<3.0.0"  # Совместимая версия pydantic
!pip install -q --upgrade aiogram  # Устанавливаем последнюю версию aiogram (совместима с pydantic 2.x)
!pip install -q nest_asyncio  # Для поддержки async в Jupyter/Colab

# Если всё ещё есть проблемы, попробуйте:
# !pip install --upgrade --force-reinstall aiogram pydantic

In [ ]:
# ============================================
# Telegram Bot интеграция (опционально)
# ============================================

# Импорты
import asyncio
import logging

# Пробуем импортировать aiogram с обработкой ошибок
try:
    from aiogram import Bot, Dispatcher, F
    from aiogram.filters import Command
    from aiogram.types import Message
    from aiogram.client.default import DefaultBotProperties
    from aiogram.enums import ParseMode
    AIOGRAM_AVAILABLE = True
except ImportError as e:
    print(f"⚠️  Ошибка импорта aiogram: {e}")
    print("💡 Попробуйте выполнить: !pip install --upgrade aiogram pydantic")
    AIOGRAM_AVAILABLE = False
    # Создаём заглушки для избежания ошибок
    Bot = None
    Dispatcher = None
    F = None
    Command = None
    Message = None
    DefaultBotProperties = None
    ParseMode = None

# Для Colab (если используется)
try:
    import nest_asyncio
    nest_asyncio.apply()
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
# ============================================
# Telegram Bot интеграция (опционально)
# ============================================

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Получение токена из Colab secrets или переменной окружения
TELEGRAM_TOKEN = None
try:
    if IN_COLAB and 'userdata' in globals():
        TELEGRAM_TOKEN = userdata.get("TELEGRAM_BOT_TOKEN")
    else:
        import os
        TELEGRAM_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
except Exception as e:
    logger.warning(f"Не удалось получить TELEGRAM_BOT_TOKEN: {e}")

# Инициализация бота (только если токен доступен)
bot = None
dp = None

if TELEGRAM_TOKEN:
    bot = Bot(token=TELEGRAM_TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
    dp = Dispatcher()
    
    # Флаг для отслеживания запуска
    bot_running = False
    
    @dp.message(Command("start"))
    async def cmd_start(message: Message):
        """Обработка команды /start"""
        await message.answer(
            "👋 Привет! Я бот для работы с Neuro Doc Assistant.\n\n"
            "📝 Просто отправьте мне ваш вопрос, и я найду ответ в документации.\n\n"
            "Команды:\n"
            "/start - начать работу\n"
            "/help - помощь\n"
            "/status - статус системы"
        )
    
    @dp.message(Command("help"))
    async def cmd_help(message: Message):
        """Обработка команды /help"""
        await message.answer(
            "ℹ️ <b>Помощь</b>\n\n"
            "Отправьте мне любой вопрос, и я:\n"
            "1. Найду релевантную информацию в документации\n"
            "2. Сгенерирую ответ на основе найденных данных\n"
            "3. Покажу метрики качества ответа\n\n"
            "Команды:\n"
            "/start - начать работу\n"
            "/help - эта справка\n"
            "/status - проверить статус системы"
        )
    
    @dp.message(Command("status"))
    async def cmd_status(message: Message):
        """Проверка статуса системы"""
        status_msg = "✅ Система готова к работе\n\n"
        
        # Проверяем наличие необходимых компонентов
        if 'run_production_pipeline' in globals():
            status_msg += "✅ Production Pipeline доступен\n"
        else:
            status_msg += "⚠️ Production Pipeline не найден\n"
        
        if 'QDRANT_URL' in globals() and QDRANT_URL:
            status_msg += "✅ Qdrant Cloud подключен\n"
        else:
            status_msg += "⚠️ Qdrant Cloud не настроен\n"
        
        await message.answer(status_msg)
    
    @dp.message(F.text & ~F.text.startswith("/"))
    async def handle_query(message: Message):
        """Обработка текстовых запросов"""
        user_query = message.text.strip()
        
        if not user_query:
            await message.answer("❌ Пожалуйста, отправьте непустой запрос.")
            return
        
        # Уведомляем о начале обработки
        processing_msg = await message.answer("⏳ Обрабатываю запрос... Это может занять некоторое время.")
        
        try:
            # Проверяем, что функция run_production_pipeline доступна
            if 'run_production_pipeline' not in globals():
                await processing_msg.edit_text(
                    "❌ Ошибка: Production Pipeline не инициализирован.\n"
                    "Пожалуйста, выполните ячейки с определением модулей."
                )
                return
            
            # Запускаем production pipeline
            result = run_production_pipeline(user_query=user_query)
            
            if result is None:
                await processing_msg.edit_text("❌ Ошибка при обработке запроса. Попробуйте еще раз.")
                return
            
            # Формируем ответ
            answer = result.get('answer', 'Ответ не получен')
            metrics = result.get('metrics', {})
            stats = result.get('statistics', {})
            
            # Основной ответ
            response_text = f"💬 <b>Ваш вопрос:</b>\n{user_query}\n\n"
            response_text += f"🤖 <b>Ответ:</b>\n{answer}\n\n"
            
            # Метрики качества
            if metrics:
                response_text += "📊 <b>Метрики качества:</b>\n"
                faithfulness = metrics.get('faithfulness', 0)
                relevancy = metrics.get('answer_relevancy', 0)
                response_text += f"  • Faithfulness: {faithfulness:.3f}"
                if faithfulness >= 0.85:
                    response_text += " ✅\n"
                else:
                    response_text += " ⚠️\n"
                response_text += f"  • Answer Relevancy: {relevancy:.3f}"
                if relevancy >= 0.80:
                    response_text += " ✅\n"
                else:
                    response_text += " ⚠️\n"
                response_text += "\n"
            
            # Статистика
            if stats:
                response_text += "📈 <b>Статистика:</b>\n"
                response_text += f"  • Найдено чанков: {stats.get('retrieved', 'N/A')}\n"
                response_text += f"  • Отобрано после reranking: {stats.get('reranked', 'N/A')}\n"
                response_text += f"  • Документов в базе: {stats.get('documents', 'N/A')}\n"
            
            # Разбиваем на части, если сообщение слишком длинное (ограничение Telegram ~4096 символов)
            max_length = 4000
            if len(response_text) > max_length:
                # Отправляем основную часть
                await processing_msg.edit_text(response_text[:max_length])
                # Отправляем остаток отдельным сообщением
                await message.answer(response_text[max_length:])
            else:
                await processing_msg.edit_text(response_text)
            
        except Exception as e:
            logger.error(f"Ошибка при обработке запроса: {e}", exc_info=True)
            await processing_msg.edit_text(
                f"❌ Произошла ошибка при обработке запроса:\n{str(e)}\n\n"
                "Попробуйте еще раз или проверьте логи."
            )
    
    async def start_bot():
        """Запуск бота"""
        global bot_running
        if bot_running:
            logger.info("Бот уже запущен")
            return
        
        if not bot or not dp:
            logger.error("Бот не инициализирован. Проверьте TELEGRAM_BOT_TOKEN")
            return
        
        try:
            bot_running = True
            logger.info("Запуск Telegram-бота...")
            await dp.start_polling(bot, skip_updates=True)
        except Exception as e:
            logger.error(f"Ошибка при запуске бота: {e}", exc_info=True)
            bot_running = False
    
    print("✅ Telegram Bot интеграция готова!")
    if TELEGRAM_TOKEN:
        print(f"✅ Токен найден. Для запуска выполните: await start_bot()")
        print(f"💡 Или используйте: asyncio.run(start_bot())")
    else:
        print("⚠️  TELEGRAM_BOT_TOKEN не найден.")
        print("💡 Для работы бота установите токен в Colab Secrets (TELEGRAM_BOT_TOKEN)")
        print("   или в переменную окружения TELEGRAM_BOT_TOKEN")
else:
    print("⚠️  Telegram Bot не инициализирован: TELEGRAM_BOT_TOKEN не найден")
    print("💡 Для работы бота установите токен в Colab Secrets (TELEGRAM_BOT_TOKEN)")
    print("   или в переменную окружения TELEGRAM_BOT_TOKEN")


### Запуск Telegram-бота

**Для запуска бота выполните следующую ячейку** (только если токен установлен):

**Важно:** Бот работает в асинхронном режиме и будет обрабатывать запросы до остановки ячейки.

In [ ]:
# ============================================
# Запуск Telegram-бота (опционально)
# ============================================

# Раскомментируйте следующую строку для запуска бота:
# await start_bot()

# Или используйте asyncio.run() для запуска вне async контекста:
# asyncio.run(start_bot())

# Для остановки бота нажмите Stop в интерфейсе Jupyter/Colab

print("💡 Для запуска бота раскомментируйте строку с await start_bot() или asyncio.run(start_bot())")
print("⚠️  Бот будет работать до остановки ячейки (Stop в интерфейсе)")
